<a href="https://colab.research.google.com/github/Thomas-Fabbris/parallel-computing-polimi/blob/main/OPENMP/OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OpenMP**
A macro-based approach for expressing thread level parallelism in C

## **Setup**

In [2]:
%%capture
!apt install build-essential libomp-dev
!mkdir /home/OpenMP
%cd /home/OpenMP

## **Glossary**

OpenMP hides from the programmer all the pedantic complexity of managing POSIX threads, deferring that to the compiler and exposing a simple set of directives/pragmas to specify how the code needs to be parallelized.
For instance, in the OpenMP model, the fork and join operations happen automatically at the stard and end of parallel constructs.
Similarly, using locks just requires denoting the critical sections of the code that must be protected by one.

Some good doc can be found here: https://rookiehpc.org/openmp/docs/index.html
<br>
Another good introduction is this on: https://github-pages.ucl.ac.uk/research-computing-with-cpp/08openmp/02_intro_openmp.html

### Pragma Syntax

OpenMP directives in C/C++ use the following general syntax (square brackets denote optional parts):

```
#pragma omp <directive> [clause[[,] clause] ...]
```

Clauses that refine the behavior of the directive to which they are applied.

Common clauses:

- `num_threads(n)` : specify the number of threads to use.
- `nowait` : remove the implicit barrier at the end of a construct.
- `if(cond)` : execute in parallel only if the condition is true.

Data sharing clauses:

- `private(varlist)` : each thread has its own uninitialized copy of listed variables.
- `firstprivate(varlist)` : like `private`, but each copy is initialized with the original value.
- `lastprivate(varlist)` : copies the value from the last iteration or section (lexicographically - w.r.t. the order as written in the code) back to the original variable; a very simple way to avoid concurrent writes.
- `shared(varlist)` : variables are shared among all threads.
- `reduction(operator : varlist)` : perform a reduction operation across threads on the given variables.
- `default(shared|private|firstprivate|lastprivate|none)` : defines the default sharing type for variables in the region; if not specified, it is `shared`; if set to `none` the compiler forces you to manually specify a sharing clause for each variable access by the thread.

Some pragmas are declarative and can sit anywhere in the serial part of the code (e.g. `omp threadprivate`), others, like worksharing ones (e.g. `omp section`), act on the block that immediately follows.
Thus, if such a pragma preceeds a control statement, it acts on its code block, otherwise a block can be induced manually with a pair of `{}`.

*Note: in C/C++ a **structured block**, is defined as a single statement or a sequence of statements that is enclosed in curly braces. It shall be such that execution may never branch into the sequence or out of it, going through it entirely after entering it. This is sometimes called "Single Entry, Single Exit" (SESE). A multi-statement block often coincides with a scope.*

Example:
```
int a = 1, b, c, d, s;
#pragma omp parallel for default(private) firstprivate(a, b) lastprivate(c) shared(s) num_threads(10)
for (int i = 0; i < 10; ++i) {
  a += 1; // everyone increments their 'a' copy from 1 to 2
  c = i;  // the master thread's 'c' will be 9 after the loop
  d = 4;  // everyone initializes its copy of 'd' to 4
  s = i;  // race condition for who will write the final 's'
}
```

Threadprivate variables:

- Declared with: `#pragma omp threadprivate(varlist)`
- Define global or file-scope variables private to each thread across parallel regions.
- Unlike `private`, their value persists across parallel regions.
- Use the `copyin(varlist)` clause to initialize threadprivate variables in each thread from the master thread.

---

### Common Pragmas

The key parallel constructs:

- `#pragma omp parallel` : start a parallel region executed by a team of multiple threads.

A team is a group of threads that work together to execute a region of code.
Each team has one master thread (the thread that encounters the parallel directive) and zero or more additional worker threads.
Within a team, threads can execute work concurrently using work-sharing directives or further nested parallel constructs.
When the parallel region ends, the team disbands, and execution continues with a single thread (the master).

Work-sharing directives:

- `#pragma omp for` : distribute loop iterations among threads in a parallel region (\*). <!--the `omp loop` pragma is its modern, more flexible, counterpart-->
- `#pragma omp sections` / `#pragma omp section` : divide work into distinct code blocks, each executed by one thread in a parallel region.

These can be combined with `omp parallel` to write just one pragma instead of two to do the same thing, like `omp parallel for` to simultaneously open a parallel region and parallelize a loop.

These pragmas terminate in an **implicit barrier** that waits for all threads to complete the work they were assigned as part of the pragma.

The parallel directive creates **one** team of threads.
Remember, **you can't "divide twice" within one team**, therefore you can't nest work-sharing directives inside the same parallel region, you can only issue them sequentially unless you create a nested parallel region that spawns additional teams.
<br>
In other words, a team can only deal with one work-sharing directive at once.

**Nesting parallel regions** provides an immediate way to allow more threads to participate in the computation.<br>
Nested parallel region behavior:
- if nested parallelism is enabled (see later), each nested region will spawn its defined number of threads each time it is encountered by any thread. This can help programs with limited scalability, but also quickly blow up the number of threads past physical cores and bloat the system (oversubscription) if abused. Enabling dynamic adjustment of the number of threads (see later) can mitigate this.
- if nested parallelism is disabled, the nested parallel region executes as if it were a serial block.

Synchronization and exclusion:

- `#pragma omp single` : only one thread executes the block; other threads wait at an implicit barrier.
- `#pragma omp master` : executed only by the master thread (thread 0), with no barrier implied.
- `#pragma omp barrier` : explicit synchronization point; all threads wait here.
- `#pragma omp critical [(name)]` : only one thread executes the block at a time; multiple critical blocks that have the same name are seen as the same block and use the same lock underneath.
- `#pragma omp atomic` : perform a single atomic update on a shared variable.
- `#pragma omp flush` : enforce memory consistency (ensures all threads see updated values).
<!--- `#pragma omp ordered` : enforce ordered execution of certain loop parts marked as `ordered`.-->

(\*) Using `omp for` requires a canonical OpenMP loop, meaning that:
- it is strictly a `for` loop;
- it has a single integer loop induction variable;
- the loop is countable (finite), with a linear increment or decrement (e.g. `i += 2` is ok, but not `i *= 2`);
- the loop variable, bounds, and increment are iteration invariants;

Following from the above, the number of iterations can be determined before the loop executes, even if it's not known until run time.
In particular, note that the number of iterations doesn't need to be known statially (at compile time). It can be computed at runtime so long as it is fixed by the time the loop is reached and needs to be divided among threads.

---

### Cancellation

OpenMP allows a thread to request that a parallel region or task region stop early, meaning the other threads should stop doing further work in that region as soon as possible.

That is `#pragma omp cancel <construct> [if-clause]`, where `<construct>` is any of: `parallel`, `sections`, `for`, `taskgroup`.

For this to work, cancellation points need to be present inside the construct, that upon reached make a thread check whether its current construct's cancellationflag was set: `#pragma omp cancellation point <construct>`.
<br>
Additionally, `barrier` and implicit barriers also act as cancellation points.

Execution reuses from immediately after the cancelled construct, either serially or by the same team depending on which was the cancelled construct.

*Note: for tasks, cancellation only acts on those grouped by the affected taskgroup construct. Tasks cancellation also occurs if the parallel region is canceled.*

---

### Scheduling

When parallelizing unbalanced loops, where some iterations may take more time than others, we can balance the load between threads with a schedule:

- `schedule(kind[, chunk_size])` : control iteration scheduling, `kind` can be:
  - `static` : iterations are split into chunks of equal size, each thread is assigned its almost evenly chunks before execution begins and those never change afterwards.
  - `dynamic` : iterations are split into chunks of equal size and placed in a queue, threads are assigned one chunk at a time from the queue. Adds a slight scheduling overhead, use only if the workload is truly unbalanced.
  - `guided` : chunks of iterations are assigned to threads as the loop runs, but the chunk size decreases as the loop progresses. The given `chunk_size` functions as the minimum chunk size reached. Also adds a slight overhead, but less than `dynamic` due to fewer scheduling decisions as per the larger initial chunks.
  - `runtime` : delegates the choice of schedule to the environment variable `OMP_SCHEDULE`.
  - `auto` : the choice of schedule is delegated to either the compiler or the runtime environment.
  - if not specified, the default schedule is implementation-dependent.
- `collapse(n)` : collapse (flatten) `n` nested loops into a single loop (and thus single ietration space) for scheduling.

Keep this in mind when parallelizing nested loops:<br>
If execution of any associated loop changes any of the values used to compute any of the **iteration counts** (loop bounds), then the behavior is unspecified.

---

### Tasks and Related Pragmas

Defines asynchronous units of work for fine-grained parallelism:

- `#pragma omp task` : define a task for deferred execution.
- `#pragma omp taskwait` : wait until all child tasks of the current task complete.
- `#pragma omp taskgroup` : group tasks for collective synchronization.
- `#pragma omp taskyield` : allow a thread to yield execution to other tasks.

Common clauses for tasks:

- `if(cond)` : create the task only if the condition is true; otherwise, execute it immediately.
- `final(cond)` : mark task as “final,” disallowing creation of child tasks inside it.
- `mergeable` : allow the task to be merged with its parent task for optimization.
- `depend(in|out|inout : varlist)` : declare task dependencies to control execution order.
- `untied` : allow the task to resume on a different thread than the one that started it.

Whereas sections define static tasks, statically defined at compile time and whose number cannot change, that are queued up and handled in arbitrary order by threads, these true **tasks** form a graph of execution with dependencies, any task spawning more tasks, and synchronization, more like threads would in a barebones fork-join model, but with cleaner code and higher level abstractions.
<br>
In brief: tasks can be spawned from any point and thread inside the parallel region!

A huge warning: tasks are tied to their thread by default, meaning that if they are suspended, they must resume in the same thread until they finish. This is crucial because private variables (e.g. those specified on the `parallel private(...)` that spawns the team of threads running the tasks) are **per-thread, not per-task**, so if a task relies on the private variables of it thread, and is untied, it may see those randomly changing if it ever gets rescheduled. And the same applies to other per-thread things, like IDs.
<br>
If you want to untie a task, make sure it only works on shared variables or its own local variables.

---

### Teams and Offloading (extra)

The `pragma opm teams` directive was introduced mainly to support heterogeneous (accelerator/GPU) programming.

It takes the place of `parallel`, but unlike it, `teams` creates multiple teams of threads, each with its own master and workers.
Each team executes the same code region independently.
Within each team, you can then launch further nested parallelism (using `omp parallel`) to create hierarchical parallelism.

On a CPU, this may just create a single team (depending on implementation), but on GPUs, **it naturally maps to CUDA thread and blocks** seen as multiple independent teams, each with their own threads.

Offloading may look like this:
```
#pragma omp target teams distribute parallel for device(device_num)
for (int i = 0; i < N; ++i) {
    A[i] = B[i] + C[i];
}
```

Where the pragma reads as:
- target : offload to a set device (e.g. GPU).
- teams : create multiple teams (like CUDA thread blocks).
- parallel for : within each team, create multiple threads to execute parts of the loop in parallel.

---

### Routines

Functions exposed by the OpenMP API:

- `int omp_get_thread_num()` : use inside a parallel region to get the unique identifier of your thread; it will be a number in [0, num_threads).
- `int omp_get_num_threads()` : returns the number of threads created in the current parallel region.
- `void omp_set_num_threads(int num_threads)` : sets the default number of threads to use in parallel regions.
- `int omp_get_max_threads()` : returns the maximum number of threads to use in parallel regions.
- `void omp_set_max_threads(int num_threads)` : sets the maximum number of threads to use in parallel regions.
- `void omp_set_nested(int nested)` : enables or disables nested parallelism.
- `void omp_set_dynamic(int dynamic_threads)` : enables or disables dynamic adjustment of the number of threads available for the execution of subsequent parallel regions.
- `double omp_get_wtime()` : returns an absolute time reference, useful to time code execution.

Additional routines are available inside the `teams` pragma:
- `int omp_get_team_num()` : returns the number of created teams.
- `int omp_get_team_num()` : returns the unique identifier of the caller thread's team; it will be a number in [0, num_teams).

To access those you need to include the `omp.h` header.

---

### Environment Variables

Control OpenMP runtime behavior without recompiling.

- `OMP_NUM_THREADS` : default number of threads to use in parallel regions, if not specified defaults to the system's core count.
- `OMP_SCHEDULE` : default loop scheduling policy (e.g. `"dynamic,4"`).
- `OMP_DYNAMIC` : allow runtime adjustment of threads (`TRUE` or `FALSE`).
- `OMP_PROC_BIND` : control thread-core binding (`master|close|spread`).
- `OMP_PLACES` : specify hardware places (`threads|cores|sockets` or custom lists).
- `OMP_MAX_ACTIVE_LEVELS` : maximum depth of nested parallel regions.
- `OMP_WAIT_POLICY` : set thread waiting behavior (`active` or `passive`).
- `OMP_DISPLAY_ENV` : print the current OpenMP environment at startup.
- `OMP_STACKSIZE` : set the thread stack size.
- `OMP_CANCELLATION` : enable or disable cancellation features in tasks or loops.
- `OMP_NESTED` : enables nested parallel regions, usually disabled by default.
- `OMP_DYNAMIC` : enables or disables dynamic adjustment of the number of threads available for the execution of subsequent parallel regions, usually disabled by default.

---

### Controlling Affinity and Thread Assignment

Mechanisms to control how threads are bound to CPU cores and how their placement affects performance:

- `proc_bind(master|close|spread)` : directive clause controlling how threads are distributed within the available places (as defined by `OMP_PLACES`):
  - `master` : all threads are placed close to the master thread (usually on the same core group, e.g. socket or NUMA node).
  - `close` : threads are packed as near as possible to each other, filling one place before moving to the next (minimize distance, maximize data locality).
  - `spread` : threads are distributed as widely as possible across places (maximize distance, maximize resource usage).

At the environment level:

- `OMP_PROC_BIND` : controls whether and how threads are bound (`TRUE|FALSE|master|close|spread`).
- `OMP_PLACES` : specifies the hardware resources threads may be placed on (e.g. `threads`, `cores`, `sockets`, or custom lists like `"{0,1},{2,3}"`).

Typical uses:
- use `close` for threads that frequently share many accesses the same data or work on contiguous chunks of a shared array, thus improving cache locality.
- use `spread` for threads that are largely independent or memory-bound, hence prefer having a lot of hardware resources and may as well fill a cache line by themselves with little data accesses in common with each other.
- with nested parallel regions, it's usual to have first a `spread` (to use different sockets or cores) and then a `close` binding (to exploit data reuse within a place), especially when inner threads see more data reuse opportunities than outer ones.

Example:

```
#pragma omp parallel num_threads(4) proc_bind(spread)
{
  #pragma omp parallel num_threads(4) proc_bind(close)
  {
    // Work here
  }
}
````

---

### Settings Precedence

When setting the same parameter through different means, they override each other in this order from most to least authoritative:

1. Explicit clauses in pragmas (`proc_bind`, `num_threads`, ...)
2. Explicit routine calls (`omp_set_num_threads`, ...)
3. Environment variables (`OMP_PROC_BIND`, `OMP_PLACES`, `OMP_NUM_THREADS`, ...)
4. Implementation defaults (compiler/runtime)

---

### Compiler Commands

Compilers (GCC, Clang) require a flag to enable OpenMP support:

```bash
gcc/clang -fopenmp program.c -o program
```

To disable OpenMP (e.g. for debugging), you can just omit `-fopenmp`.
<br>
When disabled, OpenMP pragmas are ignored, and the code runs in serial mode.
This is useful for checking correctness and debugging race conditions.

---

### Notes

* OpenMP pragmas are hints to the compiler: if support is disabled, they are ignored and the program remains valid C/C++.
* For portable and deterministic parallel programs, always explicitly specify data-sharing attributes and scheduling.

## **Simple Examples**

### **Exercise 1**

Multiple concurrent threads printing "Hello, World!" (plus meaningless computation to make it run longer):

In [ ]:
%%writefile /home/OpenMP/hello_world_0.cpp
#include <stdio.h>
#include <omp.h>

int main ()
{
  #pragma omp parallel
  {
    int id = omp_get_thread_num();
    printf("Hello World from thread = %d\n", id);
  }
}

Writing /home/OpenMP/hello_world_0.cpp


Note how this is different from the Pthreads implementation. Questions:
- How many threads will be created?
<!--implementation dependent -> usually as many as the available CPU cores-->
- What happens if you ask for a number of threads greater than the number of cores?
<!--oversubscription -> lower performance, scheduling overhead-->

Compile:

In [ ]:
!g++ hello_world_0.cpp -fopenmp -o hello_world_0

Execute:

In [ ]:
!./hello_world_0

Hello World from thread = 1
Hello World from thread = 0


### **Exercise 2**

Introducing parallelism in OpenMP can be as easy as adding pragmas, with no further modifications on the code. For example:

In [ ]:
%%writefile /home/OpenMP/hello_world.cpp

#include <stdio.h>

void print_message(int threadIndex) {
  printf("Thread number %d\n", threadIndex);
}

int main() {
  #pragma omp parallel num_threads(4)
  {
    #pragma omp for schedule(static, 4)
    for (int ii = 0; ii < 10; ii++) {
      print_message(ii);
    }
  }
  return 0;
}

Compile without the OpenMP flag:

In [ ]:
%cd /home/OpenMP
!clang hello_world.cpp -o hello_world

Inspect the generated LLVM IR:

In [ ]:
%cd /home/OpenMP
!clang hello_world.cpp -S -emit-llvm
!cat hello_world.ll

Compile with the OpenMP flag:

In [ ]:
%cd /home/OpenMP
!clang hello_world.cpp -fopenmp -o hello_world

Inspect the generated LLVM IR:

In [ ]:
%cd /home/OpenMP
!clang hello_world.cpp -S -emit-llvm -fopenmp
!cat hello_world.ll

Execute:

In [ ]:
%cd /home/OpenMP
!./hello_world

## **Calculation of pi**

###**Exercise 3**

Integral-based method to calculate pi: each thread calculates the heigth of a set of rectangles (map/SIMD pattern), the sum of all heigths is multiplied by the step size to get the area.
<img align="middle" src="https://drive.google.com/uc?id=17dBhvYY9F5Bl2re_pnmRWiZ717jolCPg">

Why does this work?
<br>
Recall that: $\frac{d}{dx} arctan(x) = \frac{1}{1+x^2}$ and $arctan(0) = 0°$ while $arctan(1) = 45° = \frac{180°}{4}$, so...

*More info here: https://math.stackexchange.com/questions/1085653/geometrical-interpretation-of-pi-int-01-frac41x2dx*

Basic implementation with manual work-sharing using thread IDs:

In [ ]:
%%writefile /home/OpenMP/integralpi.cpp
#include <stdio.h>
#include <omp.h>

#define MAX_THREADS 4

static long num_steps = 100000000;
double step;

int main() {
	int i, j;
	double pi, full_sum = 0.0;
	double start_time, run_time;
	double sum[MAX_THREADS];

	step = 1.0/(double) num_steps;

	// measure scalability from 1 to MAX_THREADS threads
	for (j = 1; j <= MAX_THREADS; j++) {
		omp_set_num_threads(j);
		full_sum = 0.0;
		start_time = omp_get_wtime();

		#pragma omp parallel
		{
			int i;
			int id = omp_get_thread_num();
			int numthreads = omp_get_num_threads();
			double x;
			sum[id] = 0.0;
			if (id == 0)
				printf(" num_threads = %d", numthreads);

			// manual work allocation
			for (i = id; i < num_steps; i += numthreads) {
				x = (i + 0.5)*step;
				sum[id] = sum[id] + 4.0/(1.0 + x*x);
			}
		}

		for(full_sum = 0.0, i = 0; i < j; i++)
			full_sum += sum[i];

		pi = step * full_sum;
		run_time = omp_get_wtime() - start_time;
		printf("\n pi is %f in %f seconds %d threads \n", pi, run_time, j);
	}
}

Questions:
- How is work distributed among threads?
<!--using thread IDs, each step every "numthreads" is assigned to a different thread-->
- Is the result deterministic?
<!--yes-->
- How do you expect performance to scale with the number of threads?
<!--ideally, linearly, as we will almost always have far more iterations to distribute than threads and there is little overhead for the creation of additional threads, aside from their creation itself; on Colab tho, anything could happen-->

Implementation with the `parallel for` work-sharing construct:

In [ ]:
%%writefile /home/OpenMP/integralpi2.cpp
#include <stdio.h>
#include <omp.h>

static long num_steps = 100000000;
double step;

int main() {
	int i;
	double x, pi, sum = 0.0;
	double start_time, run_time;

	step = 1.0/(double) num_steps;

	for (i = 1; i <= 4; i++) {
		sum = 0.0;
		omp_set_num_threads(i);
		start_time = omp_get_wtime();

		#pragma omp parallel for private(x) reduction(+:sum)
		for (i = 1; i <= num_steps; i++) {
			x = (i-0.5)*step;
			sum = sum + 4.0/(1.0+x*x);
		}

		pi = step * sum;
		run_time = omp_get_wtime() - start_time;
		printf("\n pi is %f in %f seconds and %d threads\n", pi, run_time, i);
	}
}

Questions:
- How is work distributed among threads?
<!--each thread is statically assigned some of the loop's iterations by the "for" pragma-->
- Are there other ways of resolving the access to the shared variable?
<!--yes, atomically incrementing "sum", even tho it's a poor idea performance-wise-->

## **Variables Initialization**

### **FirstPrivate**

Whenever we need to quickly give a copy of a value to each thread, we use `firstprivate`, this saves us the time needed for each thread to go and fetch a copy of an otherwise shared variable.

Say that we have an array of elements and an initial value.
We need to find subsequences of 3 contigous values in the array that such that, if added to the initial one, overflow.
We can dispatch the initial value to threads as a firstprivate variable.

In [ ]:
%%writefile /home/OpenMP/fistprivate.cpp
#include <stdio.h>
#include <omp.h>
#include <limits.h>

int main(void) {
  const int N = 12;
  unsigned int data[N] = {10, 20, 30, 250, 5, 10, 100, 200, 50, 90, 200, 40};
  unsigned int init_value = 1<<30;

  #pragma omp parallel for firstprivate(init_value)
  for (int i = 0; i < N - 2; i++) {
    unsigned int a = data[i];
    unsigned int b = data[i + 1];
    unsigned int c = data[i + 2];

    unsigned int sum = init_value;
    int overflow = 0;

    if (sum > UINT_MAX - a) overflow = 1;
    else sum += a;
    if (!overflow && sum > UINT_MAX - b) overflow = 1;
    else sum += b;
    if (!overflow && sum > UINT_MAX - c) overflow = 1;
    else sum += c;

    if (overflow)
      printf("Thread %d found overflow at subsequence [%d,%d,%d]\n", omp_get_thread_num(), i, i+1, i+2);
  }
}

Questions:
- could we do this without `firstprivate`?
<!--obviously yes, we could just have each thread copy the content of the then-shared "init_value" into its own local variable-->
- what is the advantage of using `firstprivate`?
<!--firstprivate guarantees that threads will not alter the global instance of the variable. It is mainly a semantical tool to ensure that when comparing the code between a parallel and serial execution the existence of threads doesn't unpredictably change the content of the thus-private variable. Ultimately, firstprivate clearly explicitate how the program intends the variable to be (safely) operated upon by threads: each thread receives its own copy of this initial value.-->

### **LastPrivate**

With sections (or iterations of a for loop) and `lastprivate`, we can ensure that a variable is updated by the last section (or iterations) in serial program order, not whichever finishes last in real time.

Arguably, lastprivate is very rarely useful...

In [ ]:
%%writefile /home/OpenMP/lastprivate.cpp
#include <stdio.h>
#include <omp.h>

int main() {
  int x = 0;

  #pragma omp parallel sections lastprivate(x)
  {
    #pragma omp section
    { x = 1; printf("Section 1: x=%d\n", x); }

    #pragma omp section
    { x = 2; printf("Section 2: x=%d\n", x); }

    #pragma omp section
    { x = 3; printf("Section 3: x=%d\n", x); }
  }

  printf("After sections, x=%d (from the *last* section)\n", x);
  return 0;
}

<a name="linked-list"></a>
## **Linked List Traversal**

###**Exercise 4**

Parallelizing the traversal of a linked list with OpenMP can be highly inefficient.
<img align="middle" src="https://drive.google.com/uc?id=1BrtuiwIzR2Y-xGPUt3lIX88zEfVF7e_L">
The *task* construct provides a better way to dynamically create concurrent work units.

In [ ]:
%%writefile /home/OpenMP/linkedlist.cpp
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>

struct node {
  int data;
  int fibdata;
  struct node* next;
};

struct node* init_list(struct node* p);
void processwork(struct node* p);
int fib(int n);

int fib(int n) {
  int x, y;
  if (n < 2) {
    return (n);
  } else {
    x = fib(n - 1);
    y = fib(n - 2);
    return (x + y);
  }
}

void processwork(struct node* p) {
  int n, temp;
  n = p->data;
  temp = fib(n);
  p->fibdata = temp;
}

struct node* init_list(struct node* p) {
  int i;
  struct node* head = NULL;
  struct node* temp = NULL;

  head = malloc(sizeof(struct node));
  p = head;
  p->data = 38;
  p->fibdata = 0;
  for (i = 0; i < 5; i++) {
    temp  = malloc(sizeof(struct node));
    p->next = temp;
    p = temp;
    p->data = 38 + i + 1;
    p->fibdata = i + 1;
  }
  p->next = NULL;
  return head;
}

int main() {
  double start, end;
  struct node *p=NULL;
  struct node *temp=NULL;
  struct node *head=NULL;

  printf("Process linked list\n");
  printf("  Each linked list node will be processed by function 'processwork()'\n");
  printf("  Each node will compute a subsequent Fibonacci number starting from the 38th\n");

  p = init_list(p);
  head = p;

  start = omp_get_wtime();

  #pragma omp parallel num_threads(4)
  {
    #pragma omp master
    printf("Threads: %d\n", omp_get_num_threads());
    #pragma omp single
    {
      printf("I am thread %d and I am creating tasks\n", omp_get_thread_num());
      p = head;
      while (p) {
        #pragma omp task firstprivate(p) // each task gets is own copy of the pointer to the current list node
        {
          processwork(p);
          printf("I am thread %d\n", omp_get_thread_num());
        }
        p = p->next;
      }
    }
  }

  end = omp_get_wtime();
  p = head;
  while (p != NULL) {
    printf("%d : %d\n",p->data, p->fibdata);
    temp = p->next;
    free (p);
    p = temp;
  }
  free (p);

  printf("Compute Time: %f seconds\n", end - start);
}

Questions:
- How many threads create tasks?
<!--one, it's inside the "single" pragma-->
- How are tasks distributed among threads?
<!--workpile-style, each thread fetches a new task every time it finishes the current one-->

## **Task Graphs**

Recall:
- $work$ : total amount of work
- $span$ : work on the critical path
- $parallelism = work / span$

Room for optimization:
- reducing the critical path
- reducing overhead for anything that is not on the critical path

Representation:
- nodes: tasks with a certain amount of work to do
- directed edges: dependencies between tasks (inbound arrows: what the current task needs to wait for)

For completeness, let's also recap a bit of terminology:
- when a task creates another task, the creating task becomes the *parent task* of the new task. The new task then is called a *child task* of its parent task. - the term *sibling tasks* refers to all tasks that have the same parent.
- a *descendant task* is a task in the ancestor chain of a parent, so either a child task or a task created by a descendant task (e.g., a child task of a child task).
- if a new task is put into the task pool, it is said to be *deferred* while if it is executed straight away, it is *undeferred*.
- a task is described as *completed* when it has been scheduled for execution and that execution has finished.

### **Exercise 5**

Given the following task graph:

&nbsp;

<img align="center" src="https://drive.google.com/uc?id=17PNFB2oQAFEHfvQPflSieUmjQpmLGAmn">

&nbsp;

- Calculate work and parallelism. <!--span=100+250+300, work=860, parallelism=1.32-->
- Write an OpenMP implementation reflecting the structure of the task graph.
- How many threads are active during the execution of Task 5? <!--1 or 2, depending on the state of T4-->
- Is there a better parallel implementation (considering both performance and resource usage)? I.e. do we really need to exploit all parallelism? <!--since W(T4) > W(T2) + W(T3), there is no need to run T2 and T3 in parallel (with the overhead of constructing one more thread), we could just run the sequentially T2 -> T3 and we would still be bond by the amount of work done by T4-->

### **Exercise 6**

Given the following task graph:

&nbsp;

<img align="center" src="https://drive.google.com/uc?id=15gkHj2zWAZQnuLhmPMWWSjWZqOfkj3MB">

&nbsp;

- Calculate work and span. <!--span=670, work=1045, parallelism=1.56-->
- Write an OpenMP implementation reflecting the structure of the task graph.
- Is this implementation faster than a sequential one? <!--yes, we have parallelism > 1 after all-->


### **Exercise 7**

Given the following task graph:

&nbsp;

<img align="center" src="https://drive.google.com/uc?id=1PXlO9Eaxu1lI27k2hYIaWD6y6tLZqDi9">

&nbsp;

Assume W(T1)=100, W(T2)=100, W(T3)=75, W(T4)=50, W(T5)=75, W(T6)=100, W(T7)=100, W(T8)=200.

- Calculate work and span. <!--span=400, work=800, parallelism=2-->
- Write an OpenMP implementation reflecting the structure of the task graph.
- How many threads are needed to achieve the maximum theoretical parallelism? <!--
2 threads!
For example, thread 1 handles T1, thread 2 handles T2; then thread 1 does T3, T4, and T5 while thread 2 handle T8, then thread 1 handles T6 and thread 2 finishes T7.
Note: you can always get away with "ceil(parallelism)" threads so long as you assume that a thread can "yield" a task, go do some other work, and resume it later. In this case this was not needed since luckly W(T3)+W(T4)+W(T5) = W(T8).
-->

### **Exercise 8**

Given the following task graph:

&nbsp;

<img align="center" src="https://drive.google.com/uc?id=14BBD6_ctJ-IUH1ubnaF4pMh99LNjX818">

&nbsp;

Assume W(T1)=50, W(T2)=50, W(T3)=200, W(T4)=75, W(T5)=75, W(T6)=100, W(T7)=100, W(T8)=200.

- Which dashed arrow prevents from implementing such a task structure with OpenMP? <!--none, there are still no cycles, though the dependency T6->T8 is redundant with T6->T7->T8-->
- Remove the dashed arrows, calculate work and span. <!--span=600, work=850, parallelism=1.42-->
- Write an OpenMP implementation reflecting the structure of the task graph.

### **Exercise 9**

Given the following task graph:

&nbsp;

<img align="center" src="https://drive.google.com/uc?id=1flOJlQU5-jo4kLeJmBBuou7HqYYYvQZI">

&nbsp;

Assume W(T1)=50, W(T2)=50, W(T3)=50, W(T4)=75, W(T5)=75, W(T6)=100, W(T7)=100, W(T8)=200.

<!--note the redundant dependency T3->T7-->
- Calculate work and span. <!--span=500, work=700, parallelism=1.4-->
- Write an OpenMP implementation reflecting the structure of the task graph.
- How many threads are needed to run the program? <!--ceil(1.4)=2, while T3 and T7 run by themselves in the above path, T4 can be handled by the other thread-->
- How many threads could be active during the execution of Task 3? How many during Task 5? <!--during T3, at most 2 (another on T4), during T5, at most 3 (another on T6 and another on T4)-->

## **Deadlocks 1o1**

A very simple example of how NOT to use barriers.
<br>
Also not how each thread can acquire its own ID. This is written in C++ just to spice things up.

In [ ]:
%%writefile /home/OpenMP/deadlock.cpp
#include <iostream>
#include <omp.h>

int main() {
  #pragma omp parallel default(none) shared(std::cout) num_threads(4)
  {
    const int thread_num = omp_get_thread_num();

    if(thread_num == 0) {
      std::cout << "I'm thread 0 and I caused a deadlock!" << std::endl;
    } else {
      #pragma omp barrier
    }

    #pragma omp critical
    std::cout << "I'm thread " << thread_num << std::endl;
  }
}

Overwriting /home/OpenMP/deadlock.cpp


Compile and run with a timeout (since we know it will deadlock):

In [ ]:
!g++ deadlock.cpp -fopenmp -o deadlock
!timeout 4s ./deadlock && echo "Program finished normally." || echo "Program was killed by timeout."

I'm thread 0 and I caused a deadlock!
I'm thread 0
Program was killed by timeout.


The compile often helps you, however. Something this horrific will not even compile:

In [ ]:
%%writefile /home/OpenMP/just_for_highlight.cpp
#pragma omp single
{
  std::cout << "I've caused a deadlock!\n";
  #pragma omp barrier // <-- the compiler errors out on this
}


## **Loop Schedules**

Let's see the effect of different loops schedules on an unbalanced loop nest:

In [ ]:
%%writefile /home/OpenMP/schedules.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 5000 // outer loop iterations
#define M 5000 // inner loop iterations

#define CHUNK 1000 // chunk size
#define THREADS 4  // number of threads

int main(void) {
  omp_set_nested(true); // enable nested parallelism
  omp_set_num_threads(THREADS);

  double start, end;
  double total = 0.0;

  printf("OpenMP Nested Loop Parallelization Comparison\n");
  printf("N = %d, M = %d\nCHUNK = %d, THREADS = %d \n\n", N, M, CHUNK, THREADS);

  // nested work-sharing with static schedule
  total = 0.0;
  start = omp_get_wtime();
  #pragma omp parallel for schedule(static)
  for (int i = 0; i < N; i++) {
    double local_sum = 0.0;
    // Note: we CANNOT put another work-sharing construct here like:
    // #pragma omp for schedule(static)
    // OpenMP forbids two work-sharing constructs back-to-back without a barrier or parallel pragma in-between!
    // The intended way to achieve the same result is the 'collapse' clause, see version 4.
    for (int j = 0; j < M; j++) {
      // fake unbalance: the amount of work depends on i
      for (int k = 0; k < (i % 50 + 1); k++) {
        local_sum += (i * j + k) * 1e-6;
      }
    }
    #pragma omp atomic
    total += local_sum;
  }
  end = omp_get_wtime();
  printf("Version 1 (static, nested for): %f seconds\n", end - start);

  // nested work-sharing with dynamic schedule
  total = 0.0;
  start = omp_get_wtime();
  #pragma omp parallel for schedule(dynamic)
  for (int i = 0; i < N; i++) {
    double local_sum = 0.0;
    for (int j = 0; j < M; j++) {
      for (int k = 0; k < (i % 50 + 1); k++) {
        local_sum += (i * j + k) * 1e-6;
      }
    }
    #pragma omp atomic
    total += local_sum;
  }
  end = omp_get_wtime();
  printf("Version 2 (dynamic, nested for): %f seconds\n", end - start);

  // nested parallelization with dynamic schedule
  total = 0.0;
  start = omp_get_wtime();
  #pragma omp parallel for schedule(static) reduction(+:total)
  for (int i = 0; i < N; i++) {
    // Note: here we can do this, becase we create a nested parallel region
    #pragma omp parallel for schedule(static) reduction(+:total)
    for (int j = 0; j < M; j++) {
      for (int k = 0; k < (i % 50 + 1); k++) {
        total += (i * j + k) * 1e-6;
      }
    }
  }
  end = omp_get_wtime();
  printf("Version 3 (dynamic, nested parallel for): %f seconds\n", end - start);

  // collapse clause, single parallel for with dynamic schedule
  total = 0.0;
  start = omp_get_wtime();
  #pragma omp parallel for collapse(2) schedule(dynamic, CHUNK)
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < M; j++) {
      for (int k = 0; k < (i % 50 + 1); k++) {
        total += (i * j + k) * 1e-6;
      }
    }
  }
  end = omp_get_wtime();
  printf("Version 4 (dynamic(CHUNK), collapse(2)): %f seconds\n", end - start);

  // collapse clause, single parallel for with guided schedule
  total = 0.0;
  start = omp_get_wtime();
  #pragma omp parallel for collapse(2) schedule(guided)
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < M; j++) {
      for (int k = 0; k < (i % 50 + 1); k++) {
        total += (i * j + k) * 1e-6;
      }
    }
  }
  end = omp_get_wtime();
  printf("Version 5 (guided, collapse(2)): %f seconds\n", end - start);
}


Overwriting /home/OpenMP/schedules.cpp


Compile and run:

In [ ]:
!g++ schedules.cpp -fopenmp -o schedules
!./schedules

OpenMP Nested Loop Parallelization Comparison
N = 5000, M = 5000
CHUNK = 1000, THREADS = 4 

Version 1 (static, nested for): 1.903383 seconds
Version 2 (dynamic, nested for): 1.865860 seconds
Version 3 (dynamic, nested parallel for): 3.145333 seconds
Version 4 (dynamic(CHUNK), collapse(2)): 4.494570 seconds
Version 5 (guided, collapse(2)): 4.864830 seconds


**Version 1**:
<br>
Only the outer parallel creates a team of threads.

- Low overhead, only one parallel team created.
- Static scheduling ensures predictable, reproducible thread assignment.
- Poor load balancing.

**Version 2**:
<br>
Still a single team of threads.
Dynamic scheduling lets threads grab new chunks as they finish, compensates the unbalaned workload.

- Load balancing.
- More scheduling overhead than static.

**Version 3**:
<br>
The outer parallel for creates one team of threads.
Each outer thread that hits the inner parallel for spawns a new inner team.

- Full control of both loop levels, each loop can scale and be scheduled independently.
- Can exploit more cores if your hardware and OpenMP's runtime support nested teams efficiently.
- High overhead: every outer iteration spawns a new parallel region.
- Memory and scheduling overhead outweigh benefits for small and medium-sized workloads.
- Likely to oversubscribe cores.

**Version 4**:
<br>
The two loops are merged into a single iteration space of size N*M.
One parallel team handles all (i, j) pairs directly.

- More fine grained load balancing when combined with the dynamic schedule.
- Scheduling overhead can be more effectively controlled with larger chunk sizes.
- Only works if the nested loops are perfectly nested and independent.

**Version 5**:
<br>
Same as version 4, but guided scheduling means that when there is a lot of work left to do, each thread is assigned a larger portion of it, with smaller and smaller portions being assigned as less work remains.

- Very fine grained load balancing.
- Extremely reduced scheduling overhead, as we issue more jobs only towards the end, to avoid leaving some threads at idle.

Version 5 is the cleanest and often fastest option.
<br>
*Note: this may not show on Colab, as it gives us only 2 mere cores...*

*Note: in version 1 and 2, a reduction over `local_sum` is not needed, because the inner `#pragma omp for` is not creating a new parallel region, it is simply another work-sharing construct within the same parallel team spawned by the outer pragma.*

Questions:

If you are told that exactly one iteration every 10 needs to do 5x the work, and you have thousands of iterations, what is the best schedule and why?
<!--"static", with batch multiple of 10, large enough to exploit caches, but not too large as not to cause severe unbalance in the number iterations given to each thread-->

Assume now a workload consisting of roughly 2-3x as many iterations as there will be threads (e.g. 25 iterations, 10 threads), you know that each iteration has a one-in-three chance of requiring twice the amount of work as would a normal iteration. What schedule would work best and why?
<!--"dynamic", chunks of 1, atmost 2, iterations, because with this little iterations the scheduling overhead is negligible and using larger chunks or "guided" could not effectively mitigate the unbalance due to the high likelyhood of heavier iterations-->

## **Déjà-vu**

### **Vector Product**

Just another way to see yet another reduce, but now with OpenMP pragmas!

In [8]:
%%writefile /home/OpenMP/vector_vector_prod.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

int main() {
  int N = 1000000;

  double *A = (double*) malloc(N * sizeof(double));
  double *B = (double*) malloc(N * sizeof(double));

  // initialize vectors
  for (int i = 0; i < N; i++) {
    A[i] = i * 0.001;
    B[i] = (N - i) * 0.002;
  }

  double dot = 0.0;
  double start_time = omp_get_wtime();

  // parallel reduce
  #pragma omp parallel for reduction(+ : dot) schedule(static)
  for (int i = 0; i < N; i++) {
    dot += A[i] * B[i];
  }

  double end_time = omp_get_wtime();

  printf("Dot product = %.5f\n", dot);
  printf("Computed in %.5f seconds using %d threads.\n",
  end_time - start_time, omp_get_max_threads());

  free(A);
  free(B);
}


Overwriting /home/OpenMP/vector_vector_prod.cpp


In [13]:
!g++ -fopenmp /home/OpenMP/vector_vector_prod.cpp -o /home/OpenMP/vector_vector_prod
!cd /home/OpenMP/
!./vector_vector_prod

Dot product = 333333333332.99908
Computed in 0.00843 seconds using 2 threads.


### **MatMul**

Let's just make each thread perform a vector-vector product!
<br>
However, we can try to exploit thread affinity to slightly improve cache performance by manually tiling the loop!
<br>
Say for example that we have a dual-socket motherboard with two 8-core CPUs, with each CPU itself divided in two NUMA nodes (e.g. each group of 4 cores has its own L2 cache and dedicated DRAM channel).
<br>
It's better if we split the work in 16 chunks, space far and wide 4 groups of those chunks, and then pull close the 4 chunks in each group.
This equates to cutting the output matrix's rows and cols in 4 equi-sized groups. This results in 16 chunks. Then we give each of the 4 quadrants of the output matrix to a different NUMA node and each quarter of the quadrant to a thread.

*Note: this is FAR from the best way to do a matmul on CPU! There are several improvements among which "blocking", a similar idea to tiling to better exploit caches!*
<br>
*More on this here:*
- *BLAS: https://www.netlib.org/blas/*
- *BLIS: https://dl.acm.org/doi/10.1145/2764454*
- *More on BLIS: https://dl.acm.org/doi/10.1145/2925987*


In [ ]:
%%writefile /home/OpenMP/matmul.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void init_matrix(double *M, int N, double scale) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++)
      M[i * N + j] = scale * ((i + j) % 100);
}

int main(int argc, char *argv[]) {
  int N = 1024;
  if (argc > 1) N = atoi(argv[1]);

  printf("Matrix size: %d x %d\n", N, N);

  double *A = (double*) malloc(N * N * sizeof(double));
  double *B = (double*) malloc(N * N * sizeof(double));
  double *C = (double*) calloc(N * N, sizeof(double));

  init_matrix(A, N, 0.01);
  init_matrix(B, N, 0.02);

  int groups = 4;            // outer level (NUMA groups)
  int threads_per_group = 4; // inner level threads per group
  int tile_size = N / 4;     // 4 x 4 grid of tiles

  omp_set_nested(1); // enable nested parallelism

  double t1 = omp_get_wtime();

  // outer parallel region: spread affinity
  #pragma omp parallel num_threads(groups) proc_bind(spread)
  {
    int gi = omp_get_thread_num(); // group index
    int i_start = gi * tile_size;
    int i_end   = (gi + 1) * tile_size;

    // inner parallel region: close affinity within group
    #pragma omp parallel num_threads(threads_per_group) proc_bind(close)
    {
      int gj = omp_get_thread_num(); // tile index within group
      int j_start = gj * tile_size;
      int j_end = (gj + 1) * tile_size;

      for (int i = i_start; i < i_end; i++) {
        for (int j = j_start; j < j_end; j++) {
          double sum = 0.0;
          for (int k = 0; k < N; k++) {
            sum += A[i * N + k] * B[k * N + j];
          }
          C[i * N + j] = sum;
        }
      }
    }
  }

  double t2 = omp_get_wtime();
  printf("Execution time: %.3f s\n", t2 - t1);

  free(A); free(B); free(C);
}


Writing /home/OpenMP/matmul.cpp


Compile and run:

In [ ]:
!g++ matmul.cpp -fopenmp -o matmul
!./matmul

Matrix size: 1024 x 1024
Execution time: 14.621 s


### **Histogram**

Shared memory and atomics, here we go again!
<br>
Remember to rely on privatization!

In [ ]:
%%writefile /home/OpenMP/histogram.cpp
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <omp.h>

#define NUM_LETTERS 26
#define BIN_SIZE 4
#define NUM_BINS ((NUM_LETTERS + BIN_SIZE - 1) / BIN_SIZE)

void generate_random_string(char *s, size_t len) {
  for (size_t i = 0; i < len; i++)
    s[i] = 'a' + (rand() % 26);
  s[len] = '\0';
}

int main(int argc, char *argv[]) {
  size_t N = 10000000;

  char *text = (char*) malloc(N + 1);
  srand(42);
  generate_random_string(text, N);

  // global copy
  int global_hist[NUM_BINS] = {0};

  double t1 = omp_get_wtime();

  #pragma omp parallel
  {
    // private copy
    int local_hist[NUM_BINS] = {0};

    #pragma omp for
    for (size_t i = 0; i < N; i++) {
      char c = text[i];
      if (c >= 'a' && c <= 'z') {
        int bin = (c - 'a') / BIN_SIZE;
        local_hist[bin]++;
      }
    }

    // merge private copies atomically
    for (int b = 0; b < NUM_BINS; b++) {
      #pragma omp atomic
      global_hist[b] += local_hist[b];
    }
  }

  double t2 = omp_get_wtime();
  printf("Execution time: %.4f s using %d threads\n", t2 - t1, omp_get_max_threads());

  // print histogram
  printf("\nHistogram (4-letter bins):\n");
  for (int b = 0; b < NUM_BINS; b++) {
    char start = 'a' + b * BIN_SIZE;
    char end   = start + BIN_SIZE - 1;
    if (end > 'z') end = 'z';
      printf("  %c-%c : %d\n", start, end, global_hist[b]);
  }

  free(text);
}


Overwriting /home/OpenMP/histogram.cpp


Compile and run:

In [ ]:
!g++ histogram.cpp -fopenmp -o histogram
!./histogram

Execution time: 0.0392 s using 2 threads

Histogram (4-letter bins):
  a-d : 1538630
  e-h : 1538792
  i-l : 1538822
  m-p : 1539186
  q-t : 1536136
  u-x : 1539567
  y-z : 768867


Question:
- what would be a good schedule for the parallel for? <!--static (at most guided) since all iterations are perfectly balanced, we just need to worry about maximizing the cache hits of each thread, so chunks should be reasonably larger while not becoming uneven-->
- is privatization always the best option? <!--no, say that you are counting the occurrencies of 10M items, that are uniformly distributed in the input, then atomic updates on a single shared copy could be fast enough to justify saving the cost of replicating all counters-->

## **Sections and Critical Sections**

There are two very apparent ways to optimize this code:

In [ ]:
%%writefile /home/OpenMP/two_loops.cpp
#include <cstdio>
#include <omp.h>
#include <vector>
#include <cmath>

using namespace std;

int main() {
  const int N = 10000000;
  const double target = std::sin(M_PI / 3.0);
  std::vector<double> A(N, 0.0), B(N, 0.0);
  std::vector<double> resultsA, resultsB;
  double global_sum_A = 0.0;
  double global_sum_B = 0.0;

  double start = omp_get_wtime();

  #pragma omp parallel default(none) shared(A, B, N, target, resultsA, resultsB, global_sum_A, global_sum_B)
  {
    double local_sum = 0.0;
    std::vector<double> local_vals;
    local_vals.reserve(N);

    // first loop: compute A[i]
    #pragma omp for schedule(static)
    for (int i = 0; i < N; ++i) {
      A[i] = std::sin(i * 0.001);
      local_sum += A[i];
      if (A[i] > target)
        local_vals.push_back(A[i]);
    }

    // conditional accumulation on A
    #pragma omp critical
    {
      global_sum_A += local_sum;
      resultsA.insert(resultsA.end(), local_vals.begin(), local_vals.end());
    }

    // synchronize before the next loop
    #pragma omp barrier
    local_sum = 0.0;
    local_vals.clear();

    // second loop: compute B[i]
    #pragma omp for schedule(static)
    for (int i = 0; i < N; ++i) {
      B[i] = std::cos(i * 0.001);
      local_sum += B[i];
      if (B[i] > target)
        local_vals.push_back(B[i]);
    }

    // conditional accumulation on B
    #pragma omp critical
    {
        global_sum_B += local_sum;
        resultsB.insert(resultsB.end(), local_vals.begin(), local_vals.end());
    }
  }

  double end = omp_get_wtime();

  printf("Global sums: A = %.3f, B = %.3f\n", global_sum_A, global_sum_B);
  printf("Results A:");
  for (int r = 0; r < 10; r++) printf(" %.3f", resultsA[r]);
  printf("\nResults B:");
  for (int r = 0; r < 10; r++) printf(" %.3f", resultsB[r]);
  printf("\nTotal time: %.4f seconds\n", end - start);
}

Overwriting /home/OpenMP/two_loops.cpp


Observe that the two loops (and subsequent critical sections) have no dependency with one-another, they could run concurrently!

What we need to do is:
- run each loop in its own parallel section
- remove the barrier between them
- name the two critical sections to make them independent
- now however we shared work among sections, to also share the work of each loop's iterations, we need a nested parallel region for each loop

In [ ]:
%%writefile /home/OpenMP/two_loops.cpp
#include <cstdio>
#include <omp.h>
#include <vector>
#include <cmath>

using namespace std;

int main() {
  omp_set_nested(true);

  const int N = 10000000;
  const double target = std::sin(M_PI / 3.0);
  std::vector<double> A(N, 0.0), B(N, 0.0);
  std::vector<double> resultsA, resultsB;
  double global_sum_A = 0.0;
  double global_sum_B = 0.0;

  double start = omp_get_wtime();

  #pragma omp parallel num_threads(2) default(none) shared(A, B, N, target, resultsA, resultsB, global_sum_A, global_sum_B)
  {
    #pragma omp sections
    {
      // first loop: compute A[i]
      #pragma omp section
      {
        double local_sum_A = 0.0;
        std::vector<double> local_vals_A;
        local_vals_A.reserve(N);

        #pragma omp parallel for reduction(+:local_sum_A) schedule(static)
        for (int i = 0; i < N; ++i) {
          A[i] = std::sin(i * 0.001);
          local_sum_A += A[i];
          if (A[i] > target)
            local_vals_A.push_back(A[i]);
        }

        #pragma omp critical (acc_A)
        {
          global_sum_A += local_sum_A;
          resultsA.insert(resultsA.end(), local_vals_A.begin(), local_vals_A.end());
        }
      }

      // second loop: compute B[i]
      #pragma omp section
      {
        double local_sum_B = 0.0;
        std::vector<double> local_vals_B;
        local_vals_B.reserve(N);

        #pragma omp parallel for reduction(+:local_sum_B) schedule(static)
        for (int i = 0; i < N; ++i) {
          B[i] = std::cos(i * 0.001);
          local_sum_B += B[i];
          if (B[i] > target)
            local_vals_B.push_back(B[i]);
        }

        #pragma omp critical (acc_B)
        {
          global_sum_B += local_sum_B;
          resultsB.insert(resultsB.end(), local_vals_B.begin(), local_vals_B.end());
        }
      }
    }
  }

  double end = omp_get_wtime();

  printf("Global sums: A = %.3f, B = %.3f\n", global_sum_A, global_sum_B);
  printf("Results A:");
  for (int r = 0; r < 10; r++) printf(" %.3f", resultsA[r]);
  printf("\nResults B:");
  for (int r = 0; r < 10; r++) printf(" %.3f", resultsB[r]);
  printf("\nTotal time: %.4f seconds\n", end - start);
}

Overwriting /home/OpenMP/two_loops.cpp


Compile and run:

In [ ]:
!g++ two_loops.cpp -fopenmp -o two_loops
!./two_loops

Global sums: A = 1952.308, B = -304.638
Results A: 0.866 0.867 0.867 0.868 0.868 0.869 0.869 0.870 0.870 0.871
Results B: 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
Total time: 0.5986 seconds


## **Tasks**

### **MergeSort**

Not much to say here, it's merge-sort, but with tasks...

In [ ]:
%%writefile /home/OpenMP/mergesort.cpp
#include <cstdio>
#include <cstdlib>
#include <cstring>
#include <omp.h>

// merge two sorted halves (A and B) into the output array
void merge_arrays(int* A, int nA, int* B, int nB, int* output) {
  int i = 0, j = 0, k = 0;
  while (i < nA && j < nB)
    output[k++] = (A[i] < B[j]) ? A[i++] : B[j++];
  while (i < nA) output[k++] = A[i++];
  while (j < nB) output[k++] = B[j++];
}

void merge_sort_task(int* input, int n, int* output) {
  if (n <= 2) {
    // base case: sort trivially and return
    if (n == 2 && input[0] > input[1]) {
      output[0] = input[1];
      output[1] = input[0];
    } else {
      output[0] = input[0];
      if (n == 2) output[1] = input[1];
    }
    return;
  }

  int mid = n / 2;

  // allocate space for children halves
  int* L_in  = &input[0];
  int* R_in  = &input[mid];

  int* L_out = (int*) malloc(mid * sizeof(int));
  int* R_out = (int*) malloc((n - mid) * sizeof(int));

  // spawn tasks for the left and right recursive sorts
  #pragma omp task shared(L_in, L_out)
  merge_sort_task(L_in, mid, L_out);
  #pragma omp task shared(R_in, R_out)
  merge_sort_task(R_in, n - mid, R_out);

  // wait for both halves to finish
  #pragma omp taskwait

  // merge results in "output"
  merge_arrays(L_out, mid, R_out, n - mid, output);

  free(L_out);
  free(R_out);
}

int main() {
  const int N = 64;
  int* A = (int*) malloc(N * sizeof(int));
  int* B = (int*) malloc(N * sizeof(int));

  // fill with random numbers
  for (int i = 0; i < N; i++)
  A[i] = rand() % 100;

  printf("Original array:\n");
  for (int i = 0; i < N; i++) printf("%d ", A[i]);
    printf("\n\n");

  double t1 = omp_get_wtime();

  #pragma omp parallel
  {
    #pragma omp single
    merge_sort_task(A, N, B);
  }

  double t2 = omp_get_wtime();

  printf("Sorted array:\n");
  for (int i = 0; i < N; i++) printf("%d ", B[i]);
  printf("\n");

  printf("\nExecution time: %.6f s\n", t2 - t1);

  free(A);
  free(B);
  return 0;
}

Overwriting /home/OpenMP/mergesort.cpp


Compile and run:

In [ ]:
!g++ mergesort.cpp -fopenmp -o mergesort
!./mergesort

Original array:
83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 

Sorted array:
2 5 11 11 13 15 15 19 21 21 22 23 24 25 26 26 26 27 27 29 29 29 30 35 35 36 37 40 42 49 56 56 58 59 62 62 62 63 67 67 67 68 69 70 70 72 73 73 77 80 81 82 83 84 84 86 86 90 91 92 93 93 96 98 

Execution time: 0.000426 s


### **Algebraic Expression Evaluation**

Let's evaluate this linar algebra expression with OpenMP tasks:

$A, B, C, D, E \in \mathbb{R}^{4 \times 4}$

$A \cdot (A \cdot B - C \cdot D) - E^2 \cdot D^2$

In [ ]:
%%writefile /home/OpenMP/algebra.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 4

void matmul(double A[N][N], double B[N][N], double C[N][N]) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++) {
      double s = 0.0;
      for (int k = 0; k < N; k++) s += A[i][k] * B[k][j];
      C[i][j] = s;
    }
}

// alpha = +1 => add, alpha = -1 => sub
void matadd(double A[N][N], double B[N][N], double C[N][N], double alpha) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++)
      C[i][j] = A[i][j] + alpha * B[i][j];
}

void init(double M[N][N], double scale) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++)
      M[i][j] = scale * ((i + j) % 7);
}

void printmat(const char *name, double M[N][N]) {
  printf("%s:\n", name);
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++)
      printf("%6.2f ", M[i][j]);
    printf("\n");
  }
}

int main() {
  double A[N][N], B[N][N], C[N][N], D[N][N], E[N][N];
  double AB[N][N], CD[N][N], inner[N][N], Aterm[N][N];
  double E2[N][N], D2[N][N], E2D2[N][N], Result[N][N];

  init(A, 0.5); init(B, 0.7); init(C, 0.9);
  init(D, 1.1); init(E, 1.3);

  double t1 = omp_get_wtime();

  #pragma omp parallel
  #pragma omp single
  {
    // AB = A*B
    #pragma omp task depend(out:AB)
    matmul(A, B, AB);

    // CD = C*D
    #pragma omp task depend(out:CD)
    matmul(C, D, CD);

    // inner = AB - CD
    #pragma omp task depend(in:AB, CD) depend(out:inner)
    matadd(AB, CD, inner, -1.0);

    // Aterm = A * inner
    #pragma omp task depend(in:A, inner) depend(out:Aterm)
    matmul(A, inner, Aterm);

    // E2 = E*E
    #pragma omp task depend(out:E2)
    matmul(E, E, E2);

    // D2 = D*D
    #pragma omp task depend(out:D2)
    matmul(D, D, D2);

    // E2D2 = E2 * D2
    #pragma omp task depend(in:E2, D2) depend(out:E2D2)
    matmul(E2, D2, E2D2);

    // Result = Aterm - E2D2
    #pragma omp task depend(in:Aterm, E2D2) depend(out:Result)
    matadd(Aterm, E2D2, Result, -1.0);

    #pragma omp taskwait
  }

  double t2 = omp_get_wtime();
  printf("Execution time: %.3f ms\n", 1e3 * (t2 - t1));

  printmat("Result", Result);
  return 0;
}


Compile and run:

In [ ]:
!g++ algebra.cpp -fopenmp -o algebra
!./algebra

### **Sum of Products**

Assume you are given strings repersenting algebraic expressions in the form of a sums of products with only 2 possible variable, "a" and "b", like:
<br>
"a+a\*b\*b+b\*b\*a\*b\*a+b\*b\*b"

The following program parses the expression and dynamically spawns an OpenMP task to solve each product, then reducing sequentially (an alternative implementation could have each task di an atomic add to the global sum).
<br>
This scheme comes in handy when the input is very long and you would rather not have to iterate over it more than once, but you can still identify suitable independente (and thus parallelizable) units of work as you go.

In [ ]:
%%writefile /home/OpenMP/sop.cpp
#include <stdio.h>
#include <iostream>
#include <stdlib.h>
#include <string.h>
#include <omp.h>

#define N_TERMS 64

// evaluate a product term like "a*b*b*a"
float eval_product(const char *term, float a, float b) {
  float res = 1.0f;
  for (const char *p = term; *p; p++) {
    if (*p == 'a') res *= a;
    else if (*p == 'b') res *= b;
  }
  return res;
}

int main(int argc, char* argv[]) {
  if (argc < 2) {
      std::cerr << "Usage: " << argv[0] << " <expression>\n";
      return 1;
  }

  const char* expr = argv[1]; // e.g. "a*b*b*a+a*a+b*b"
  float a = 2.0f, b = 3.0f;

  printf("Expression: %s\n", expr);
  printf("a = %.2f, b = %.2f\n", a, b);

  char *expr_copy = strdup(expr); // duplicate the string
  char *terms[N_TERMS]; // array of pointers to product terms
  int n_terms = 0;
  // split the expression at every '+', isolating product terms
  for (char *tok = strtok(expr_copy, "+"); tok; tok = strtok(NULL, "+"))
    terms[n_terms++] = tok;

  float partials[N_TERMS] = {0.0f};
  float result = 0.0f;

  double t1 = omp_get_wtime();

  #pragma omp parallel
  #pragma omp single
  {
    // spawn a parsing task that spawns product tasks
    #pragma omp task depend(out:partials)
    {
      for (int i = 0; i < n_terms; i++) {
        int idx = i;
        const char *term = terms[i];
        #pragma omp task firstprivate(idx, term) depend(out:partials[idx])
        {
          partials[idx] = eval_product(term, a, b);
          //printf("Term %d (%s) = %.2f\n", idx, term, partials[idx]);
        }
      }
    }

    // reduction task that depends on all partials
    #pragma omp task depend(in:partials) depend(out:result)
    {
      float sum = 0.0f;
      for (int i = 0; i < n_terms; i++) sum += partials[i];
      result = sum;
    }

    #pragma omp taskwait
  }

  double t2 = omp_get_wtime();

  printf("Result = %.2f\n", result);
  printf("Execution time: %.3f ms\n", 1e3 * (t2 - t1));

  free(expr_copy);
  return 0;
}


Overwriting /home/OpenMP/sop.cpp


Compile and run:

In [ ]:
!g++ sop.cpp -fopenmp -o sop
!./sop a*b*b*a+a*a+b*b+a*b*b*a*a*a+b*b*b*b*b+a*b*a*a*b*a+a*a+b*b*a*a*a

Expression: a*b*b*a+a*a+b*b+a*b*b*a*a*a+b*b*b*b*b+a*b*a*a*b*a+a*a+b*b*a*a*a
a = 2.00, b = 3.00
Result = 0.00
Execution time: 0.146 ms


### **MatMul with Tasks**

Here we see a slightly new pragma, `taskloop`: splits the iteration space of a loop into OpenMP tasks.
While this looks similar to the `for` worksharing construct, the behavior is fundamentally different.
When using the `for` construct, all threads in the parallel region have to encounter the construct so that they can split up the work, whereas the taskloop construct needs only be executed by a single thread (e.g. the master thread).

*Note: the taskloop construct is defined in a way that is similar to the definition of a regular task.
This means that if N threads encounter the construct, each of the threads will start executing the same loop, so the loop will be executed N times rather than having a single incarnation of the loop split between them.*

The `grainsize` clause defines how many iterations should be executed per task.
Funnily enough, the OpenMP standard allows this to be an interval, which in this example will be eight to sixteen iterations, so an implementation has some flexibility in choosing the exact number of iterations (and, therefore, tasks).
The construct also supports the `num_tasks` clause, which specifies exactly how many tasks should be created, and then adjusts the chunk size accordingly.

In [ ]:
%%writefile /home/OpenMP/tmatmul.cpp
#include <cstdio>
#include <cstdlib>
#include <omp.h>

void matmul_taskloop(float *C, const float *A, const float *B, size_t n) {
  #pragma omp parallel firstprivate(n)
  {
    #pragma omp master
    {
      // we could also add "collapse(3)" if we had >n HW threads
      #pragma omp taskloop firstprivate(n) grainsize(8)
      for (int i = 0; i < n; ++i)
        for (int k = 0; k < n; ++k)
          for (int j = 0; j < n; ++j)
            C[i * n + j] += A[i * n + k] * B[k * n + j];
    }
  }
}

void init_matrix(float *M, int n, float scale) {
  for (int i = 0; i < n; i++)
    for (int j = 0; j < n; j++)
      M[i*n + j] = scale * float((i + j) % 13);
}

int main() {
  int n = 512;

  printf("Matrix size: %d x %d\n", n, n);

  float *A = (float*) aligned_alloc(64, n * n * sizeof(float));
  float *B = (float*) aligned_alloc(64, n * n * sizeof(float));
  float *C = (float*) aligned_alloc(64, n * n * sizeof(float));

  init_matrix(A, n, 0.01f);
  init_matrix(B, n, 0.02f);
  init_matrix(C, n, 0.0f);

  double t1 = omp_get_wtime();
  matmul_taskloop(C, A, B, n);
  double t2 = omp_get_wtime();

  printf("Time: %.3f sec\n", t2 - t1);

  free(A);
  free(B);
  free(C);
}


Overwriting /home/OpenMP/tmatmul.cpp


Compile and run:

In [ ]:
!g++ tmatmul.cpp -fopenmp -o tmatmul
!./tmatmul

Matrix size: 512 x 512
Time: 0.707 sec


## **Cancellation**

Very useful for programs that "work in parallel towards an objective". And upon the achievement of the objective by on threads, others end their purpose too.

In this simple example, we search for a specific value in an array. Upon a thread finding it, cancellation allows the whole parallel region to end cleanly.

In [ ]:
%%writefile /home/OpenMP/cancellation.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 1000000

int main() {
  const int target = 777;
  int found_index = -1;

  int data[N];
  for (int i = 0; i < N; ++i)
    data[i] = 1;

  data[543210] = target; // hide the target somewhere

  // parallel search with cancellation
  #pragma omp parallel shared(found_index)
  {
    #pragma omp for
    for (int i = 0; i < N; ++i) {

      if (data[i] == target) {
        found_index = i;
        // stop everyone
        #pragma omp cancel for
      }

      #pragma omp cancellation point for
    }
  }

  printf("Found target at index: %d\n", found_index);

  return 0;
}

Overwriting /home/OpenMP/cancellation.cpp


Compile and run:

In [ ]:
!g++ cancellation.cpp -fopenmp -o cancellation
!./cancellation

Found target at index: 543210


<a name="loop-dependencies"></a>
## **Loop Dependencies**

Loops are always the easiest, and most beneficial, target for optimization!
That is all well and good so long as each iteration is independent of the others. However, start considering inter-iteration (loop-carried) data dependencies, and there may be an hard limit on the parallelism that a loopnest can exploit.

These exercises focus on identifying such data dependecies between iterations and thus inferring the highest degree of concurrency existing in a loopnest.

How to:
- identify the indices accessed by each iterations
- identify the memory locations read and written by each iteration
- see if subsequent iterations cause a pattern of reads occurring where there previously was a write (RAW), writing something that was read previously (WAR), written something twice but with different values (WAW)
- measure the distance between the each of the above reads and writes
- how those patterns would unfold sequentially is your ground truth
- you can go parallel only so long as the relative order of operations in each dependency is preserved

*Another way to view loops with inter-iteraiton dependencies: try to find a window of (often contigous, but not necessarily) iterations that can run in parallel. Such a window is valid iff no data dependencie outgoing from its iterations lands back in the window itself.*

### **Exercise 10**

In [ ]:
%%writefile highlight_me.cpp

for (j = 10; j < 100; j++)
    for (i = 10; i < 100; i++)
        a[i][j] += f(a[i][j - 10]);

How and under what conditions can the following code be parallelized? Is there a way to write the code for better parallelization?
<!--
As of now, the "i" loop can run fully in parallel for every iteration of "j".
Every batch of 10 consecutive iterations on "j" can run concurrently, as there is a loop-carried dependency W(j)(a)->R(j + 10)(a) [write now, read 10 js after].

Yes, we can split the "j" loop to explicitate the dependency:
for (j_d = 1; j_d < 10; j_d++)
    for (j_p = 0; j_p < 10; j_p++) // can run fully in parallel now
        int j = j_d*10 + j_p;
        for (i = 10; i < 100; i++)
            a[i][j] += f(a[i][j - 10]);
-->

### **Exercise 11**

In [ ]:
%%writefile highlight_me.cpp

for (i = 4; i < 100; i++) {
    for (j = 1; j < 100; j++)
        a[i][0] *= f(a[i][j]);
    for (j = 0; j < 100; j++)
        b[i - 4][j] += f(b[i][j]);
}

How and under what conditions can the following code be parallelized? Is there a way to write the code for better parallelization? Assume that `a` and `b` store integers.
<!--
Note that "a" only depends on "a" and "b" only on "b". Therefore an independent "i" loop could be built for each of them.

Then, the "j" loop on "a" is essentially performing a product-based reduction of all elements of "a" for a given "i". So that can be easily parallelized, e.g. by following a binary reduction tree. While the "i" loop on "a" can run fully in parallel.

For "b", instead, each "i" iteration depends on the "i-4" one, therefore we can at best run 4 iterations in parallel by splitting the loops into two.
Finally, "b"'s "j" loop can run fully in parallel for each "i", as there are no dependencies between iterations.

Final version:
for (i = 4; i < 100; i++)
    // e.g. omp for reduce(*:a[i][0])
    for (j = 1; j < 100; j++)
        a[i][0] *= f(a[i][j]);

for (i_d = 1; i_d < 25; i_d++)
    for (i_p = 0; i_p < 4; i_p++)
        int i = i_d*4 + i_p;
        for (j = 0; j < 100; j++)
            b[i-4][j] += f(b[i][j]);
-->

### **Exercise 12**

In [ ]:
%%writefile highlight_me.cpp

for (j = 0; j < 999; j++)
    for (i = 2; i < 999; i++)
        a[i][j] = f(a[i - 2][j]);

Evaluate the cache-friendliness of this code and consider its relationship to parallelizability. How does the memory layout of the array affect performance, and what changes could improve cache utilization vs parallelizability? Justify your answer. Assume that `a` stores integers.
<!--
Assuming "a" to be stored in row-major, we have that elements with consecutive "j"s are contigous in memory, while between consecutive "i"s there is a huge gap. Therefore, one innermost iteration, that is on "i", accesses two elements quite far apart, the next one jumps to two further different locations, and only the next one again has "i-2" landing close to where the first iteration wrote, and so on. This repeats for all "j"s.
In terms of performance, at regime, this would cause one every two memory accesses to be a cache miss.
Solution:
Either store "a" in col-major or iterate on "i" first.

For parallelizability, two iterations of "i" could run in parallel, therefore we could break the "i" loop into two.
The "j" loop is instead fully parallelizable.


One possible final version:
for (i_d = 1; i_d < 499; i_d++)
    for (i_p = 0; i_p < 2; i_p++)
        int i = i_d*2 + i_p;
        for (j = 0; j < 999; j++)
            a[i][j] = f(a[i - 2][j]);
-->

### **Exercise 13**

In [ ]:
%%writefile highlight_me.cpp

for (i = 0; i < 100; i++)
  for (j = 5; j < 200; j++)
    a[i][j] = f(a[i][j - 5], b[i + 1][j]);

Under what conditions can the code be parallelized? Is there a way to restructure the loops to expose more parallelism?

There is a loop-carried dependency with distance 5 on "j": W(j)(a)->R(j + 5)(a).
Therefore only 5 iterations on "j" can run in parallel at once.
All iterations on "i" can instead run in parallel.
As for "b", it carries no dependencies, as it is never written.

We can rewrite the loopnest as follows to explose more parallelism:
for (i = 0; i < 100; i++) // fully parallel
  for (j_d = 1; j_d < 40; j_d++)
    for (j_p = 0; j_p < 5; j_p++) // now those can run in parallel
      int j = j_d * 5 + j_p;
      a[i][j] = g(a[i][j - 5], b[i + 1][j]);

### **Exercise 14**

In [ ]:
%%writefile highlight_me.cpp

// assume "a" already initialized to some useful data
for (i = 0; i < 100; i++)
  for (j = 1; j < 100; j++)
    a[i][j] = f(a[i][j - 1], a[i + 1][j]);

Determine true data dependencies. Is either loop safe to parallelize? Is there any transformation that could expose more parallelism?

Dependencies:
W(i)(j)(a) -> R(i)(j + 1)(a) // RAW leftward
R(i)(j)(a) -> W(i + 1)(j)(a) // WAR downward

In words: "to compute my element, I need the original, untouched, one below me, and the already computed on on my left".

This forms a tight "dependencies diamond", for which loop parallelization (over "i" or "j") is impossible. And even if the offsets were slightly more than "1"s, very little could be achieved.

However, we CAN parallelize over each diagonal! Since all elements on each diagonal are fully independent!
We can iterate over a diagonal at a time, top-left towards the bottom-right, and inside each diagonal all computations can run in parallel because:
- each diagonal only sees each row once, top to bottom, therefore the WAR is satisfied
- each diagonal only sees each column once, left to right, therefore the RAW is satisfied

Possible rewrite:
for (int d = 1; d <= 198; d++) // sequential: one diagonal at a time
  for (int i = 0; i <= d; i++) // can be fully parallel: inside the diagonal
    int j = d - i;
    if (i >= 0 && i < 100 && j >= 1 && j < 100 && i + 1 < 100)
      a[i][j] = f(a[i][j - 1], a[i + 1][j]);

Only drawback: the number of the elements on each diagonal varies! Therefore each outer iteration has a different number of possibly parallel inner iterations...

## **SoA vs AoS**

This is a good momento to pause and reflect on access patterns and cache locality...

<br>

Example of when **AoS (Array of Structures)** wins:

You repeatedly iterate over all fields of one object at a time, e.g., computing the Euclidean length of each 3D vector.
<br>
Good spatial locality is achieved if all elements of each vector are close together.

In [ ]:
%%writefile /home/OpenMP/highlight_only.cpp
#include <vector>
#include <cmath>
#include <iostream>

struct Vec3 {
  float x, y, z;
};

// this computation uses all three components per element -> keep them close to each other to localize accesses
void compute_lengths(const std::vector<Vec3> &points, std::vector<float> &out) {
  size_t N = points.size();
  for (size_t i = 0; i < N; ++i) {
    const Vec3 &p = points[i];
    out[i] = std::sqrt(p.x*p.x + p.y*p.y + p.z*p.z);
  }
}

int main() {
  // AoS
  std::vector<Vec3> pts(1'000'000, {1,2,3});
  std::vector<float> lengths(pts.size());
  compute_lengths(pts, lengths);
  std::cout << "First length = " << lengths[0] << "\n";
}


Why this is optimal (AoS):
- we repeatedly load x, y, and z of the same object
- with AoS, these are near each other, leading to fewer cache misses
- SIMD compilers can load {x, y, z} as a single vector load
- prefetching becomes trivial

Using SoA instead would cause:
- three independent large arrays to fetch
- more translation lookaside buffer (TLB) pressure
- more cache bandwidth needed (streams from 3 arrays)

Example of when **SoA (Structure of Arrays)** wins:

You repeatedly iterate over one field across all objects, e.g. updating only the x coordinates of many points.
Good spatial locality is achieved if elements accessed in quick succession are nearby.

In [ ]:
%%writefile /home/OpenMP/highlight_only.cpp
#include <vector>
#include <iostream>

// SoA
struct PointsSoA {
  std::vector<float> x, y, z;
  PointsSoA(size_t N) : x(N), y(N), z(N) {}
};

// this computation uses only one field -> keep is successive values close for locality
void update_x(PointsSoA &pts) {
  size_t N = pts.x.size();
  for (size_t i = 0; i < N; ++i) {
    pts.x[i] += 1.0f;
  }
}

int main() {
  PointsSoA pts(1'000'000);
  update_x(pts);
  std::cout << "First x = " << pts.x[0] << "\n";
}


Why this is optimal (SoA):
- the loop reads/modifies only the x array
- memory access is perfectly sequential
- compilers generate SIMD code easily for sequential accesses
- zero waste: no need to touch y or z

AoS here would cause:
- hardware reads fill cache lines with {x, y, z} even though you only need x
- you waste 2/3 of bandwidth
- lower memory throughput
- harder for the compiler to auto-vectorize (strided loads instead of contiguous)

## **Parallel Patterns**

Some of these examples were originally from this [notebook](https://colab.research.google.com/drive/1guQZSGxxDmSizKv4cHd-H0WpgpA4aw9I?usp=sharing).
<br>
And since that notebook was written in C++, we are gonna stick with C++ for a bit here as well.

Write the following file before compiling any of the examples below.
<br>
This is just a basic helper to time the various patterns.

In [ ]:
%%writefile /home/OpenMP/timer.hpp
#ifndef TIMER_HDR
#define TIMER_HDR

#include <chrono>
#include <iostream>
#include <string>

template <class clock_type = std::chrono::high_resolution_clock> class timer {
  using time_point = std::chrono::time_point<clock_type>;
  time_point _start;
  std::string _name;

public:
  timer(std::string name = "")
      : _start(clock_type::now()), _name(std::move(name)) {}
  ~timer() {
    const auto diff = std::chrono::duration_cast<std::chrono::milliseconds>(
        clock_type::now() - _start);
    std::cerr << "[ function " << _name << " took " << diff.count() << "ms ]"
              << std::endl;
  }
};

template <class callable_type, class... arguments_type>
inline auto timer_print(std::string name, callable_type &&function,
                        arguments_type &&...args) {
  timer t(std::move(name));
  return function(args...);
}

#define time(name, ...) timer_print(#name, name, __VA_ARGS__)

#endif // TIMER_HDR

Writing /home/OpenMP/timer.hpp


General rule of thum:
- MAP: each element updates independently of all others
- GATHER: each element compute the location where to read (in other words, threads sit on the sender side)
- SCATTER: each element compute the location where to write (in other words, threads sit on the receiver side)

### **Map**

This application implement the *SAXPY* : Single Precision A times X plus Y, which is the computation `z[i] = a*x[i] + y[i]` with `x, y, z` three vectors of equal length.

Clearly, this can be seen as a MAP pattern from each element of `x` and `y` to one of `z`, and parallelized as such.

After recognizing the pattern, a MAP is trivial to parallelize in OpenMP, almost always merely requiring a `parallel for`.

In [ ]:
%%writefile /home/OpenMP/map.cpp
#include <algorithm>
#include <array>
#include <cstdlib>
#include <iostream>
#include <random>

#include "timer.hpp"

// define the data structures
static constexpr auto vector_size = std::size_t{10000};
std::array<float, vector_size> x, y, z;

// define a random generator
float random_float() {
  static std::mt19937 random_generator{2023};
  std::uniform_real_distribution<float> interval(float{0.0}, float{1.0});
  return interval(random_generator);
}

// define the computation [mapped] function
void saxpy(std::array<float, vector_size> &z, const float a, const std::array<float, vector_size> &x, const std::array<float, vector_size> &y) {
  #pragma omp parallel for
  for (std::size_t i = 0; i < vector_size; ++i) {
    z[i] = a * x[i] + y[i];
  }
}

int main() {
  // initialize the input data
  const float a = 4.87;
  std::generate(std::begin(x), std::end(x), random_float);
  std::generate(std::begin(y), std::end(y), random_float);
  std::fill(std::begin(z), std::end(z), float{0});

  // perform the computation
  time(saxpy, z, a, x, y);

  // print the output
  std::cout << "z = {";
  if (vector_size > std::size_t{0}) {
    std::cout << z[0];
  }
  for (std::size_t i = 1; i < std::min(std::size_t{10}, vector_size); ++i) {
    std::cout << ", " << z[i];
  }
  if (vector_size > std::size_t{10}) {
    std::cout << ", ...";
  }
  std::cout << "}" << std::endl;

  // if we reach this point, everything is fine
  return EXIT_SUCCESS;
}

Writing /home/OpenMP/map.cpp


Compile and run:

In [ ]:
!g++ map.cpp -fopenmp -o map
!./map

[ function saxpy took 0ms ]
z = {2.48043, 3.70224, 4.61079, 3.09507, 3.03533, 4.25946, 1.59451, 0.476707, 1.19255, 4.47768, ...}


### **Map Reduce**

In this example, we need to compute the [Shannon Entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of a large-ish dataset.

Given an array of length $N$, containing integers in the interval $\{0, \dots K - 1\}$, such entropy is defined as:
$$
H = -\sum_{i=0}^{K-1} p_i \cdot log_2(p_i) \text{ and } p_i = \frac{count_i}{N}.
$$
Where $count_i$ is the number of occurencies of integer $i$ in the above interval, and thus $p_i$ is the estimated probability with which $i$ occurs in the array.

For this example, assume $K$ is small, such as $K < 16$.
<br>
We can compute all $count_i$-s as an histogram (SCATTER-REDUCE) and then compute the Shannon entropy (MAP-REDUCE).
<br>
Specifically, using privatization for the histogram, threads map (scatter) each array value to its bin, then all private copies are reduced over into the global one.
Then, a map-reduce can be performed to compute $H$'s sum, mapping each histogram counter to $p_i \cdot log_2(p_i)$, with each thread accumulating a local partial sum, then all threads reducing to the final global sum for $H$.

*Note: do not confuse the binning, with incremented counters, of the histogram, with the BIN pattern. The BIN patter starts by building an histogram as well, but then proceeds with a PACK by bucket.*

In [ ]:
%%writefile /home/OpenMP/map_reduce.cpp
#include <algorithm>
#include <cmath>
#include <cstdlib>
#include <iostream>
#include <random>

#include "timer.hpp"

static constexpr std::size_t N = 50000;
static constexpr std::size_t K = 12;

int *data;
std::size_t hist[K];

// random integer generator in [0, K)
int random_int() {
  static std::mt19937 rng{2025};
  std::uniform_int_distribution<int> dist(0, K - 1);
  return dist(rng);
}

// HISTOGRAM with PRIVATIZATION and SCATTER-REDUCE
void histogram_map_reduce(std::size_t hist[K], const int *data) {
  for (std::size_t i = 0; i < K; ++i) hist[i] = 0;

  #pragma omp parallel for reduction(+: hist[:K])
  for (std::size_t i = 0; i < N; ++i) {
    hist[data[i]]++;
  }
}

// ENTROPY with MAP-REDUCE
double entropy_map_reduce(const std::size_t hist[K]) {
  double Nf = double(N);
  double H = 0.0;

  #pragma omp parallel for reduction(+:H)
  for (std::size_t k = 0; k < K; ++k) {
    if (hist[k] == 0) continue;
    double p = hist[k] / Nf;
    H += -p * std::log2(p);
  }

  return H;
}

int main() {
  data = new int[N];
  std::generate(data, data + N, random_int);

  // PHASE 1: HISTOGRAM
  time(histogram_map_reduce, hist, data);

  // PHASE 2: ENTROPY
  double H = time(entropy_map_reduce, hist);

  std::cout << "Histogram: ";
  for (std::size_t k = 0; k < K; ++k)
    std::cout << hist[k] << " ";
  std::cout << "\nEntropy H = " << H << " bits\n";

  delete[] data;
  return EXIT_SUCCESS;
}

Overwriting /home/OpenMP/map_reduce.cpp


Compile and run:

In [ ]:
!g++ map_reduce.cpp -fopenmp -o map_reduce
!./map_reduce

[ function histogram_map_reduce took 0ms ]
[ function entropy_map_reduce took 0ms ]
Histogram: 4011 4183 4138 4095 4180 4289 4147 4225 4174 4175 4242 4141 
Entropy H = 3.58477 bits


### **Workpile**

Here we generalize the map pattern to a scenario where the total number of tasks (applications of the function) is not known a priori.
In particular, we consider a tree search:

<img src="https://i.imgur.com/WHBRp2C.png">

This code first constructs a balanced binary tree with the values [1, 200].
Then it searches a value in the tree.
<br>
The tree search is carried out by a finite team of threads via tasks.
First, one thread searches the root.
Then, as a thread does not find the value and needs to prosecute down multiple branches, its spawns a new task per-branch.
This way tasks, representing yet-to-visit branches, build up a WORKPILE that threads progressively work through.

Possible improvements:
- as soon as a task finds the value, set a flag and dispose of all other tasks immediately.
- build a binary search tree (BST) instead

In [ ]:
%%writefile /home/OpenMP/workpile.cpp
#include <memory>
#include <iostream>
#include <string>

struct node {
  inline node(const int v) : value(v) {}
  int value;
  std::unique_ptr<node> left = nullptr;
  std::unique_ptr<node> right = nullptr;
};

static void print_internal(std::unique_ptr<node> &root, std::ostream &out) {
  if (root) {  // we have a valid tree
    if (root->left) {
      out << '\t' << root->value << " -> " << root->left->value << std::endl;
    }
    if (root->right) {
      out << '\t' << root->value << " -> " << root->right->value << std::endl;
    }
    if (root->left || root->left) {  // we have some child
      print_internal(root->left, out);
      print_internal(root->right, out);
    } else {  // we don't have any child
      out << '\t' << root->value << std::endl;
    }
  } else {  // this is an empty leaf
  }
}

void print(std::unique_ptr<node> &root, std::ostream &out) {
  out << "digraph {" << std::endl;
  print_internal(root, out);
  out << "}" << std::endl;
}

// functions to manipulate the tree

static void insert(std::unique_ptr<node> &root, const int value) {
  if (root) {  // we have a valid tree (continue to change it)
    //if (value < root->value) { // this would yield a BST
    if (std::rand() & 1) {
      if (root->left) {
        insert(root->left, value);
      } else {
        root->left = std::make_unique<node>(value);
      }
    } else if (value > root->value) {
      if (root->right) {
        insert(root->right, value);
      } else {
        root->right = std::make_unique<node>(value);
      }
    } else {
      // we have a duplicate value, no need to do anything
    }
  } else {  // we don't have a valid tree as input (a new one)
    root = std::make_unique<node>(value);
  }
}

void populate(std::unique_ptr<node> &root, const int min_value, const int max_value) {
  if (min_value < max_value) {  // we are building the tree branches
    const int range = min_value + max_value;
    const int middle_value = range / 2;
    insert(root, middle_value);
    populate(root, min_value, middle_value - 1);
    populate(root, middle_value + 1, max_value);
  } else if (min_value == max_value - 1) {  // leaf with two values
    insert(root, min_value);
    insert(root, max_value);
  } else if (min_value == max_value) {  // leaf with just one value
    insert(root, min_value);
  }
}

// function applied over the tree

std::string path_builder(const node* root, const int value, const std::string partial_result) {
  if (root != nullptr) {
    if (value == root->value) {  // we found the actual value
      return partial_result + std::to_string(root->value);
    } else {  // we need to look on both ways
      std::string left, right;
      if (root->left) {
        #pragma omp task shared(left)
        left = path_builder(root->left.get(), value, partial_result + std::to_string(root->value) + " -> ");
      }
      if (root->right) {
        #pragma omp task shared(right)
        right = path_builder(root->right.get(), value, partial_result + std::to_string(root->value) + " -> ");
      }
      #pragma omp taskwait
      return !left.empty() ? left : right;
    }
  } else {
    return "";
  }
}

std::string pathfinder(const node* root, const int value) {
  std::string result;
  #pragma omp parallel
  {
    #pragma omp single
    {
      result = path_builder(root, value, std::string{});
    }
  }
  return result;
}

int main() {
  // initialize the tree
  std::unique_ptr<node> tree;
  populate(tree, 1, 200);
  print(tree, std::cerr);

  // find an element in the tree
  const int target_value = 187;
  std::cout << "Path to " << target_value << ": " << pathfinder(tree.get(), target_value) << std::endl;

  return EXIT_SUCCESS;
}

Writing /home/OpenMP/workpile.cpp


Compile and run:

In [ ]:
!g++ workpile.cpp -fopenmp -o workpile
!./workpile

digraph {
	100 -> 50
	100 -> 125
	50 -> 12
	50 -> 62
	12 -> 13
	12 -> 45
	13 -> 20
	13 -> 21
	20 -> 60
	20 -> 61
	60
	61 -> 111
	61 -> 73
	111 -> 178
	111 -> 159
	178
	159
	73 -> 144
	73 -> 165
	144 -> 151
	144 -> 175
	151
	175
	165 -> 176
	165
	21 -> 34
	21 -> 66
	34 -> 123
	34 -> 35
	123 -> 171
	171
	35 -> 58
	35 -> 55
	58
	55
	66 -> 79
	66 -> 86
	79 -> 91
	79
	86
	45 -> 68
	45 -> 52
	68 -> 120
	68 -> 126
	120 -> 162
	162 -> 182
	162
	126
	52 -> 63
	52 -> 150
	63 -> 82
	63 -> 95
	82 -> 108
	82 -> 152
	108 -> 174
	174
	152
	95 -> 118
	95
	150 -> 141
	150 -> 157
	141 -> 177
	177
	157
	62 -> 71
	62 -> 85
	71 -> 77
	71 -> 74
	77 -> 84
	77 -> 80
	84 -> 135
	84 -> 112
	135
	112 -> 196
	196
	80 -> 153
	80 -> 192
	153 -> 164
	164
	192 -> 193
	192
	74 -> 92
	74 -> 110
	92 -> 170
	92 -> 185
	170
	185
	110 -> 131
	110
	85 -> 158
	85 -> 109
	158 -> 167
	167 -> 188
	188
	109 -> 121
	109 -> 124
	121 -> 154
	121 -> 140
	154
	140 -> 163
	163 -> 187
	163
	124 -> 143
	124
	125 -> 106
	125 -> 137
	106 

### **Scatter and Gather**

What is a GATHER if not a MAP pattern whose function is a memory **read**.
<br>
What is a SCATTER if not a MAP pattern whose function is a memory **write** (with a side-dish of race conditions).

An obvious example: the histogram pattern with privatization! (yes, again XD)

In [ ]:
%%writefile /home/OpenMP/gather_scatter.cpp
#include <iostream>
#include <vector>
#include <random>
#include <omp.h>

int main() {
  const int N = 10000;
  const int K = 16;

  std::vector<int> data(N);
  std::vector<int> global_hist(K, 0);

  // init. input
  std::mt19937 gen(2024);
  std::uniform_int_distribution<int> dist(0, K - 1);
  for (int &v : data) v = dist(gen);

  #pragma omp parallel
  {
    // thread-private histogram
    std::vector<int> local_hist(K, 0);

    // GATHER: each thread reads its part of the input
    #pragma omp for
    for (int i = 0; i < N; ++i) {
      int v = data[i];
      local_hist[v]++;
    }

    // SCATTER REDUCTION: merge private hists into the global one
    for (int k = 0; k < K; ++k) {
      #pragma opm atomic
      global_hist[k] += local_hist[k];
    }
  }

  std::cout << "Histogram counts:\n";
  for (int k = 0; k < K; ++k)
  std::cout << "bin " << k << " : " << global_hist[k] << "\n";

  return EXIT_SUCCESS;
}

Writing /home/OpenMP/gather_scatter.cpp


Compile and run:

In [ ]:
!g++ gather_scatter.cpp -fopenmp -o gather_scatter
!./gather_scatter

Histogram counts:
bin 0 : 622
bin 1 : 596
bin 2 : 658
bin 3 : 619
bin 4 : 647
bin 5 : 601
bin 6 : 645
bin 7 : 640
bin 8 : 604
bin 9 : 635
bin 10 : 620
bin 11 : 621
bin 12 : 600
bin 13 : 619
bin 14 : 623
bin 15 : 650


### **Permute**

What is constructing an array's permutation in parallel if not a SCATTER!
<br>
And not any SCATTER, if, by construction, there never is a collision, there are no data races to worry about.

For an example of this, let's "decrypt" a message with a simple encoding:
<br>
Split the message into even-indexed and odd-indexed characters
```
even_part = message[0], message[2], message[4], …
odd_part = message[1], message[3], message[5], …
```
then concatenate them
```
shuffled = even_part + odd_part
```
finally perform a cyclic left shift by K positions
```
scrambled[i] = shuffled[(i + k) mod N]
```
The results is a "not very much readable" a permutation of the original message.

Decoding such messages naturally maps onto the PERMUTE (scatter) pattern!

*Possible improvement: could do the un-rotate and un-concatenate in one single permute!*

In [ ]:
%%writefile /home/OpenMP/permute.cpp
#include <iostream>
#include <string>
#include <vector>
#include <omp.h>

#include "timer.hpp"

// -----------------------------------------------------------
// ENCODING (done offline):
//   1) split message into even-index chars and odd-index chars
//   2) concatenate evens first, then odd ones ==> interleave shuffle
//   3) left-rotate the resulting string by K positions
//
// DECODING (done here):
//   1) right-rotate the input string by K
//   2) SCATTER each character to its original index
// -----------------------------------------------------------

std::string rotate_right(const std::string& s, int k) {
  int n = s.size();
  k = ((k % n) + n) % n;
  return s.substr(n - k) + s.substr(0, n - k);
}

std::string unscramble_interleave(const std::string& enc) {
  int n = enc.size();
  std::string out(n, '?');

  int num_evens = (n + 1) / 2; // ceil(n/2)

  // PERMUTE (SCATTER)
  #pragma omp parallel for
  for (int i = 0; i < n; ++i) {
    if (i < num_evens) { // even pos
      int orig = 2 * i;
      out[orig] = enc[i];
    } else { // odd pos
      int j = i - num_evens;
      int orig = 2 * j + 1;
      out[orig] = enc[i];
    }
  }

  return out;
}

int main() {
  std::string original = "OPENMP MAKES ME FEEL LIKE I HAVE CLONES";

  int k = 7;

  // encoding (done sequentially)
  std::string enc1;
  std::string ev, od;
  for (int i = 0; i < (int)original.size(); ++i)
      (i % 2 == 0 ? ev : od).push_back(original[i]);
  enc1 = ev + od;
  std::string scrambled = enc1.substr(k) + enc1.substr(0, k);

  std::cout << "Scrambled message:\n" << scrambled << "\n\n";

  // decode
  std::string rotated_back = rotate_right(scrambled, k);
  std::string recovered = time(unscramble_interleave, rotated_back);

  std::cout << "Recovered message:\n" << recovered << "\n";

  return 0;
}


Overwriting /home/OpenMP/permute.cpp


Compile and run:

In [ ]:
!g++ permute.cpp -fopenmp -o permute
!./permute

Scrambled message:
EFE IEIHV LNSPNPMKSM ELLK  AECOEOEM AE 

[ function unscramble_interleave took 0ms ]
Recovered message:
OPENMP MAKES ME FEEL LIKE I HAVE CLONES


### **Gather and Scatter - a practical use case**

Let's take a look at a classic, slightly simplified, computer-vision pipeline for edge-detection consisting of two phases:

1) Gradient magnitude computation.
Each pixel reads its neighbors to compute local intensity changes. <br>GATHER: a single output location gathers multiple input values.
It is fully parallel and conflict-free.
Could also be seen as a 4-point STENCIL.

2) [Hough transform](https://en.wikipedia.org/wiki/Hough_transform) line voting.
Each detected edge pixel casts multiple "votes" into an accumulator array,
marking all lines that could pass through it. <br>SCATTER: one input produces many outputs, and many inputs write to the same accumulator cells.
Requires atomic updates or privatization.

Variations of this pipeline come up often in computer vision, doing something like:
<br>
input image -> edge map -> line accumulator -> line detection

In [ ]:
%%writefile /home/OpenMP/edge_detection.cpp
#include <vector>
#include <cmath>
#include <iostream>
#include <iomanip>
#include <omp.h>

#include "timer.hpp"

// gradient magnitude (GATHER): each output pixel reads neighbors to estimate an edge strength
void compute_gradient(const std::vector<float> &img, int H, int W, std::vector<float> &grad) {
  #pragma omp parallel for collapse(2)
  for (int i = 1; i < H - 1; i++) {
    for (int j = 1; j < W - 1; j++) {
      // 4-point stencil
      float gx = img[i * W + (j + 1)] - img[i * W + (j - 1)];
      float gy = img[(i + 1) * W + j] - img[(i - 1) * W + j];
      grad[i * W + j] = std::sqrt(gx * gx + gy * gy);
    }
  }
}

// Hough voting (SCATTER): each strong edge pixel scatters votes into a global accumulator
void hough_transform(const std::vector<float> &grad, int H, int W, std::vector<int> &acc, int num_rho, int max_rho, int num_theta) {
  float dtheta = M_PI / num_theta;

  #pragma omp parallel for collapse(2)
  for (int i = 0; i < H; i++) {
    for (int j = 0; j < W; j++) {

      if (grad[i * W + j] < 50.0f)
        continue; // ignore weak pixels

      // scatter votes
      for (int t = 0; t < num_theta; t++) {
        float theta = t * dtheta;
        float rho = j * std::cos(theta) + i * std::sin(theta);
        int r = int(rho + max_rho);

        #pragma omp atomic
        acc[t * num_rho + r]++;
      }
    }
  }
}

int main() {
  const int H = 128, W = 128;

  std::vector<float> img(H * W, 0.0f);
  for (int i = 0; i < H; i++)
    img[i * W + i] = 255.0f; // bright diagonal

  std::cout << "Image:\n";
  for (int i = 0; i < H && i < 20; i++) {
    for (int j = 0; j < W && j < 20; j++)
      std::cout << std::setw(5) << std::fixed << std::setprecision(2) << std::right << img[i*W + j] << " ";
    std::cout << "\n";
  }

  std::vector<float> grad(H * W, 0.0f);
  time(compute_gradient, img, H, W, grad);

  float max_rho = std::sqrt(H * H + W * W);
  int num_rho = 2 * int(max_rho);
  const int num_theta = 180;
  std::vector<int> acc;
  acc.assign(num_theta * num_rho, 0);
  time(hough_transform, grad, H, W, acc, num_rho, max_rho, num_theta);

  std::cout << "Votes:\n";
  for (int i = 0; i < num_rho && i < 20; i++) {
    for (int j = 0; j < num_theta && j < 20; j++)
      std::cout << std::setw(5) << std::fixed << std::setprecision(2) << std::right << acc[i*num_theta + j] << " ";
    std::cout << "\n";
  }
}


Overwriting /home/OpenMP/edge_detection.cpp


Compile and run:

In [ ]:
!g++ edge_detection.cpp -fopenmp -o edge_detection
!./edge_detection

Image:
255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00 255.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00

*What does this transform actually do?*

The goal of the Hough transform is to detect straight lines in an image.
A line can be represented in "normal form":
$$
\rho = x \cdot cos\theta + y \cdot sin\theta
$$
Where $\theta$ is the angle of the line's normal (0 to $\pi$), $\rho$ is the perpendicular distance from the origin to the line, and $x, y$ the pixel coordinates.

From this, each line in the image corresponds to one point in the ($\theta$, $\rho$) space.

In practice, the code, for each strong (high-gradient) edge pixel at ($x$, $y$),loops over all possible $\theta$s, e.g., 0°, 1°, 2°, ... 179°.
<br>
For each it computes $\rho = x \cdot cos\theta + y \cdot sin\theta$ and goes to increment the ($\theta$, $\rho$) accumulator.


*How does this detect lines?*

Consider all edge pixels on a real line L in the image.
For the $\theta$ corresponding to L's orientation:
- their computed $\rho$ values will be the same
- they all vote into the same accumulator

Hence, that accumulator becomes larger than the rest, with such a peak revealing the presence (and parameters) of the line.

<br>

For us:
- GATHER for the gradients;
- SCATTER from image-space ($x$, $y$) to ($\theta$, $\rho$) space;

### **Zip and Permute**

Given three same-length vectors of integers (SoA form) we want to convert it into a single contigous array with one tripled of values after the other, each tripled having a value per same index position in the original array.
Then, XOR each value with a given key and internally sort each triplet.

We start with the three arrays of equal length, our job is to look at each group of three elements in the same position in the three arrays as a single tuple.
Therefore, we ZIP (GATHER) the arrays into one, alternatingly picking elements from the first, second, and third array.
<br>
Subsequently, after each element is XORed with the fixed integer (MAP), we need to ensure that each tuple of three elements that we zipped together is internally sorted (lowest element comes first). This is done by a PERMUTE (SCATTER).

In short:
<br>
ZIP (into tuples) -> MAP (xor) -> PERMUTE (sort each tuple)

In [ ]:
%%writefile /home/OpenMP/zip_permute.cpp
#include <iostream>
#include <vector>
#include <random>
#include <omp.h>

#include "timer.hpp"

int main() {
  const int N = 20;
  const int XOR_VALUE = 0x5A5A;

  // init. input arrays
  std::vector<int> A(N), B(N), C(N);

  std::mt19937 gen(42);
  std::uniform_int_distribution<int> dist(1, 100);
  for (int i = 0; i < N; ++i) {
    A[i] = dist(gen);
    B[i] = dist(gen);
    C[i] = dist(gen);
  }

  // zipped array of tuples, Z[3*i] = A[i], Z[3*i + 1] = B[i], Z[3*i + 2] = C[i]
  std::vector<int> Zipped(3 * N);

  // ZIP (GATHER)
  #pragma omp parallel for
  for (int i = 0; i < N; ++i) {
    Zipped[3*i] = A[i];
    Zipped[3*i + 1] = B[i];
    Zipped[3*i + 2] = C[i];
  }

  // MAP: XOR each element in the zipped array
  #pragma omp parallel for
  for (int i = 0; i < 3*N; ++i) {
    Zipped[i] ^= XOR_VALUE;
  }

  // PERMUTE (SCATTER) each tuple
  #pragma omp parallel for
  for (int i = 0; i < N; ++i) {
    int a = Zipped[3*i];
    int b = Zipped[3*i + 1];
    int c = Zipped[3*i + 2];

    // scatter permutation into correct locations (manual binary comparisons tree)
    if (b < a) {
        if (c < b) {
            Zipped[3*i] = c; Zipped[3*i + 1] = b; Zipped[3*i + 2] = a;
        } else if (c < a) {
            Zipped[3*i] = b; Zipped[3*i + 1] = c; Zipped[3*i + 2] = a;
        } else {
            Zipped[3*i] = b; Zipped[3*i + 1] = a; Zipped[3*i + 2] = c;
        }
    } else {
        if (c < a) {
            Zipped[3*i] = c; Zipped[3*i + 1] = a; Zipped[3*i + 2] = b;
        } else if (c < b) {
            Zipped[3*i] = a; Zipped[3*i + 1] = c; Zipped[3*i + 2] = b;
        } else {
            Zipped[3*i] = a; Zipped[3*i + 1] = b; Zipped[3*i + 2] = c;
        }
    }
  }

  std::cout << "Final permuted tuples:\n";
  for (int i = 0; i < N; ++i) {
    std::cout << "(" << Zipped[3*i] << ", " << Zipped[3*i+1] << ", " << Zipped[3*i+2] << ") ";
  }
  std::cout << "\n";

  return 0;
}

Overwriting /home/OpenMP/zip_permute.cpp


Compile and run:

In [ ]:
!g++ zip_permute.cpp -fopenmp -o zip_permute
!./zip_permute

Final permuted tuples:
(23050, 23098, 23164) (23056, 23060, 23113) (23114, 23142, 23142) (23114, 23120, 23159) (23053, 23132, 23156) (23125, 23143, 23160) (23064, 23069, 23129) (23059, 23099, 23132) (23044, 23054, 23116) (23102, 23113, 23131) (23109, 23113, 23140) (23131, 23140, 23151) (23108, 23129, 23158) (23140, 23151, 23154) (23108, 23124, 23135) (23096, 23106, 23167) (23061, 23120, 23156) (23118, 23140, 23165) (23097, 23142, 23150) (23052, 23135, 23157) 


### **Scan**

This is the classic prefix sums or SCAN.
<br>
Implemented here in the classical Blelloch fashion and adapted to OpenMP with explicit tiles.

In [ ]:
%%writefile /home/OpenMP/scan.cpp
#include <omp.h>
#include <algorithm>
#include <array>
#include <cstdlib>
#include <iostream>
#include <random>
#include <vector>

#include "timer.hpp"

// data structures
static constexpr auto vector_size = std::size_t{100000};
std::array<int, vector_size> x, cumulative;

// random generator for initialization
int random_int() {
  static std::mt19937 random_generator{2023};
  std::uniform_int_distribution<int> interval(0, 100);
  return interval(random_generator);
}

// computation function -> scan (inclusive, out-of-place)
void kernel(std::array<int, vector_size> &cumulative, const std::array<int, vector_size> &x) {
  static_assert(vector_size > std::size_t{0});
  cumulative[0] = x[0];

  // temporary array to store intermediate results
  std::vector<int> intermediate_results;

  #pragma omp parallel
    {
    // identify the tile size (elements per thread)
    const std::size_t num_tiles = static_cast<std::size_t>(::omp_get_num_threads());
    const std::size_t tile_size = (vector_size - std::size_t{1}) / num_tiles;

    // -> allocate the intermeadiate results array
    // intermediate_results[i] = prefix sum up to, and exclusing, tile "i"
    #pragma omp single
    {
      intermediate_results.resize(num_tiles, int{0});
      intermediate_results[0] = x[0]; // hard-coded
    }

    // -> reduction phase of the algorithm
    // reduce sequentially inside each tile, and in parallel among them
    #pragma omp for
    for (std::size_t tile_index = 1; tile_index < num_tiles; ++tile_index) {
      const std::size_t start_index = (tile_index - std::size_t{1}) * tile_size + std::size_t{1};
      const std::size_t stop_index = start_index + tile_size;
      for (std::size_t i = start_index; i < stop_index; ++i) {
        intermediate_results[tile_index] += x[i];
      }
    }

    // -> sequential scan among tiles reduction results
    #pragma omp single
    {
      for (std::size_t i = std::size_t{1}; i < num_tiles; ++i) {
        intermediate_results[i] += intermediate_results[i - std::size_t{1}];
      }
    }

    // -> perform the "missing operations" phase of the algorithm
    // prefix sum sequentially inside each tile, and in parallel among them
    // add the reduced sum from the tile before yours at the start
    #pragma omp for
    for (std::size_t tile_index = 0; tile_index < num_tiles; ++tile_index) {
      const std::size_t start_index = (tile_index * tile_size) + std::size_t{1};
      const std::size_t stop_index = start_index + tile_size;
      cumulative[start_index] = intermediate_results[tile_index] + x[start_index];
      for (std::size_t i = start_index + std::size_t{1}; i < stop_index; ++i) {
        cumulative[i] = cumulative[i - std::size_t{1}] + x[i];
      }
    }

    // take care of the leftovers
    const std::size_t start_index = num_tiles * tile_size + std::size_t{1};
    #pragma omp single
    for (std::size_t i = start_index; i < vector_size; ++i) {
      cumulative[i] = cumulative[i - std::size_t{1}] + x[i];
    }
  }
}

int main() {
  // initialize the input data
  std::generate(std::begin(x), std::end(x), random_int);
  std::fill(std::begin(cumulative), std::end(cumulative), int{0});

  // perform the computation
  kernel(cumulative, x);

  // print the last ten values
  std::cout << "cumulative = {";
  if (vector_size > std::size_t{11}) {
    std::cout << "...";
    for (std::size_t i = vector_size - 11; i < vector_size; ++i) {
      std::cout << ", " << cumulative[i];
    }
  } else {
    std::cout << cumulative[0];
    for (std::size_t i = 1; i < vector_size; ++i) {
      std::cout << ", " << cumulative[i];
    }
  }
  std::cout << "}" << std::endl;

  return EXIT_SUCCESS;
}

Writing /home/OpenMP/scan.cpp


Compile and run:

In [ ]:
!g++ scan.cpp -fopenmp -o scan
!./scan

cumulative = {..., 5007296, 5007342, 5007380, 5007462, 5007491, 5007564, 5007596, 5007679, 5007718, 5007742, 5007759}


### **Counting Sort**

Another use of our good old friend the histogram.

Non-comparative sorting algorithms don't have the "n*logn" limit, but need to leverage some assumption on the input. In this case, we need to sort an array containing only numbers in $\{0, \dots 99\}$.

GATHER for the histogram -> SCAN over bins to get output offsets -> SCATTER to permute the input into the output

This sequence of GATHER->SCAN->SCATTER is the **BIN** parallel pattern!
<br>
The BIN parallel pattern is a two-phase counting-and-placement operation over a collection of data elements, where:
- first, each element is mapped to a discrete category (bin)
- then, all elements belonging to the same category are grouped together in the output

*Note: SCATTER race conditions prevented by keeping an atomically incremented counter per BIN indicating where to insert its next element.*

In [ ]:
%%writefile /home/OpenMP/counting_sort.cpp
#include <vector>
#include <iostream>
#include <random>
#include <algorithm>
#include <omp.h>

#include "timer.hpp"

static constexpr int N = 40; // input length
static constexpr int K = 100; // input integers are in [0, K - 1]

void counting_sort(const std::vector<int> &input, std::vector<int> &output) {
  const size_t N = input.size();
  output.resize(N);

  // global histogram
  std::vector<int> global_bins(K, 0);

  // SCATTER
  // count the occurrencies of each value
  #pragma omp parallel
  {
    // thread-private histogram
    std::vector<int> local_bins(K, 0);

    #pragma omp for
    for (size_t i = 0; i < N; ++i) {
      int v = input[i];
      ++local_bins[v];
    }

    // a tiny SCATTER to finish the historam
    for (int k = 0; k < K; ++k) {
      #pragma omp atomic
      global_bins[k] += local_bins[k];
    }
  }

  // SCAN (prefix sum)
  // compute the starting idx of each distinct value
  // note: done serially here, to keep the code clean,
  //       there is parallel example above anyway
  std::vector<int> offsets(K);
  int running = 0;
  for (int k = 0; k < K; ++k) {
    offsets[k] = running;
    running += global_bins[k];
  }

  std::vector<int> global_pos = offsets;

  // SCATTER
  // place each value in its final position
  #pragma omp parallel for
  for (size_t i = 0; i < N; ++i) {
    int v = input[i];
    // same as atomic "int pos = global_pos[v]++;"
    // practically: reserve the next position in the bin
    int pos = __atomic_fetch_add(&global_pos[v], 1, __ATOMIC_RELAXED);
    output[pos] = v;
  }
}

int main() {
  std::mt19937 rng(2025);
  std::uniform_int_distribution<int> dist(0, 99);

  std::vector<int> input(N);
  for (auto &x : input)
  x = dist(rng);

  std::vector<int> sorted;
  time(counting_sort, input, sorted);

  std::cout << "Input: ";
  for (int v : input) std::cout << v << " ";
  std::cout << "\n";

  std::cout << "Sorted: ";
  for (int v : sorted) std::cout << v << " ";
  std::cout << "\n";

  return 0;
}

Writing /home/OpenMP/counting_sort.cpp


Compile and run:

In [ ]:
!g++ counting_sort.cpp -fopenmp -o counting_sort
!./counting_sort

[ function counting_sort took 9ms ]
Input: 13 49 88 34 93 10 44 64 38 92 25 28 65 78 49 12 96 88 80 38 45 15 80 37 4 52 76 36 0 8 29 33 61 73 91 7 30 7 24 19 
Sorted: 0 4 7 7 8 10 12 13 15 19 24 25 28 29 30 33 34 36 37 38 38 44 45 49 49 52 61 64 65 73 76 78 80 80 88 88 91 92 93 96 


### **Radix Sort**

We can take counting sort up a notch and remove its fatal flaw of depending on the number of distinct entries in the array by determining ourself what those distinct entries are.

The idea behind radix sort is to only look at the lowest R of bits of each value, sort w.r.t. them, by using them as bins for counting sort, and then repeat the process for the next R lowest bit. If the sorting algorithm used underneath is **stable** (preserves relative order of equivalent elements), iterating leads to a final sorted array. All while dealing with exactly $2^{\text{R}}$ distinct elements and no more at each round.

For each round, the pattern is the same as counting sort, that is, the BIN:
<br>
GATHER for the radix histogram -> SCAN for offsets -> SCATTER to sort -> repeat after shifting the bitmask
<br>
From the above, we require this BIN pattern to be stable.

*Note: if R is 1, thus we sort by just one bit at a time, the BIN collapses into the SPLIT pattern!*

In [ ]:
%%writefile /home/OpenMP/radix_sort.cpp
#include <vector>
#include <iostream>
#include <random>
#include <algorithm>
#include <omp.h>

#include "timer.hpp"

static constexpr int N = 40; // input length

constexpr int RADIX_BITS = 4; // setting to 1 collapses the BIN pattern in to a SPLIT
constexpr int RADIX = 1 << RADIX_BITS; // count of possible distinct within the radix
constexpr unsigned int MASK = RADIX - 1; // 0x00..FF

void radix_sort(std::vector<unsigned>& a) {
  const int n = a.size();
  // ping-pong copy of the array
  std::vector<unsigned> tmp(n);

  int num_threads = omp_get_max_threads();
  const int passes = (32 + RADIX_BITS - 1) / RADIX_BITS;

  // global histogram
  std::vector<int> global_bins(RADIX, 0);
  // bins[thread][bucket] -> local histogram copies per thread, one bin per different radix
  std::vector<std::vector<int>> bins(num_threads, std::vector<int>(RADIX));
  // thread_offsets[thread][bucket]
  std::vector<std::vector<int>> thread_offsets(num_threads, std::vector<int>(RADIX));
  // offsets[b] -> starting index where to store the content of bucket b
  std::vector<int> offsets(RADIX, 0);

  #pragma omp parallel
  {
    int tid = omp_get_thread_num();
    for (int pass = 0; pass < passes; pass++) {
      unsigned shift = pass * RADIX_BITS;

      std::fill(global_bins.begin(), global_bins.end(), 0);
      std::fill(bins[tid].begin(), bins[tid].end(), 0);
      std::fill(offsets.begin(), offsets.end(), 0);

      // classic parallel histogram: each threed sees a part of the array
      #pragma omp for schedule(static)
      for (int i = 0; i < n; i++) {
        unsigned digit = (a[i] >> shift) & MASK;
        bins[tid][digit]++;
      }

      // one thread merges the local copies and builds global offsets
      #pragma omp single
      {
        // merge private histograms
        // NOTE: as alwasy, could be done in parallel with REDUCE or ATOMICS
        for (int t = 0; t < num_threads; t++)
          for (int b = 0; b < RADIX; b++)
            global_bins[b] += bins[t][b];

        // compute where each bin can be stored from its size
        // NOTE: could be done with a parallel SCAN
        for (int b = 1; b < RADIX; b++)
          offsets[b] = offsets[b - 1] + global_bins[b - 1];

        // assign to each thread the offset for its bins w.r.t. the global offset of each bin
        for (int b = 0; b < RADIX; b++) {
          int offset = offsets[b];
          for (int t = 0; t < num_threads; t++) {
            thread_offsets[t][b] = offset;
            offset += bins[t][b];
          }
        }
      }

      // SCATTER from each thread's portion of the array to their computed offsets
      #pragma omp for schedule(static)
      for (int i = 0; i < n; i++) {
        unsigned digit = (a[i] >> shift) & MASK;
        int pos = thread_offsets[tid][digit]++;
        tmp[pos] = a[i];
      }

      // copy back the written buffer over the original one
      #pragma omp for schedule(static)
      for (int i = 0; i < n; i++)
        a[i] = tmp[i];

      #pragma omp barrier
    }
  }
}

int main() {
  std::mt19937 rng(2025);
  std::uniform_int_distribution<unsigned> dist(0, 1 << 24);

  std::vector<unsigned> input(N);
  for (auto &x : input)
  x = dist(rng);

  time(radix_sort, input);

  std::cout << "Input: ";
  for (unsigned v : input) std::cout << v << " ";
  std::cout << "\n";

  std::cout << "Sorted: ";
  for (unsigned v : input) std::cout << v << " ";
  std::cout << "\n";

  return 0;
}

Overwriting /home/OpenMP/radix_sort.cpp


Compile and run:

In [ ]:
!g++ radix_sort.cpp -fopenmp -o radix_sort
!./radix_sort

[ function radix_sort took 0ms ]
Input: 53202 699911 1267256 1312747 1444825 1770878 2120783 2273114 2590756 3319207 4170793 4321751 4815406 4912526 5035094 5669727 5727787 6093619 6348579 6379525 6513511 7475393 7637077 8264741 8384383 8866734 10249442 10864917 11028798 12300798 12909361 13132676 13438290 13439525 14798413 14895680 15318058 15573585 15646527 16177237 
Sorted: 53202 699911 1267256 1312747 1444825 1770878 2120783 2273114 2590756 3319207 4170793 4321751 4815406 4912526 5035094 5669727 5727787 6093619 6348579 6379525 6513511 7475393 7637077 8264741 8384383 8866734 10249442 10864917 11028798 12300798 12909361 13132676 13438290 13439525 14798413 14895680 15318058 15573585 15646527 16177237 


### **Filter and Pack**

You are given an array of integers.
<br>
You need to construct an array containing only those integers that are divisible by 2 in the original.
Crucially, the relative order of array elements must be preserved.

The slow way to do this: go in parallel over the array and append survivors to a thread-private list. Then, in increasing order of thread-id, concatenate the private lists into the final one.

The fast way to do this: parallel MAP to filter the array, counting how many "to keep" ones each thread has seen, and replacing others with -1. Parallel SCAN to provide each thread with the number of elements that were kept before his own (essentially, the offset where to write its results). Parallel PACK to put together all values.

*Note: it is often the case for PACK to rely on SCAN (exclusive) to compute offsets and coordinate writes!*

In [ ]:
%%writefile /home/OpenMP/filter_pack.cpp
#include <vector>
#include <iostream>
#include <omp.h>

#define N 1000000

int main() {
  std::vector<int> in(N);
  for (int i = 0; i < N; i++)
    in[i] = i;

  std::vector<int> out;

  // per-thread counts and offsets (built from counts after the SCAN)
  std::vector<int> counts;
  std::vector<int> offsets;

  #pragma omp parallel
  {
    int tid = omp_get_thread_num();
    int nth = omp_get_num_threads();

    #pragma omp single
    {
      counts.resize(nth);
      offsets.resize(nth);
    }

    // manual work allocation between threads
    int chunk = (int)((N + nth - 1) / nth);
    int begin = tid * chunk;
    int end = std::min(N, begin + chunk);

    // MAP
    int local_keep = 0;
    for (int i = begin; i < end; i++) {
      if (in[i] % 2 == 0)
        local_keep++;
      else
        in[i] = -1;
    }
    counts[tid] = local_keep;

    #pragma omp barrier

    // INCLUSIVE SCAN (PREFIX-SUM)
    // NOTE: to keep the code brief, this is a work-inefficient (n^2) version with a ton of barriers!
    for (int step = 1; step < nth; step *= 2) {
      int add = 0;
      if (tid >= step)
        add = counts[tid - step];

      #pragma omp barrier

      offsets[tid] = counts[tid] + add;

      #pragma omp barrier

      counts[tid] = offsets[tid];
    }

    #pragma omp barrier

    // make the SCAN EXCLUSIVE
    offsets[tid] -= local_keep;

    // let the last thread resize the output
    if (tid == nth - 1)
      out.resize(offsets[tid] + local_keep);

    #pragma omp barrier

    // PACK
    int outpos = offsets[tid];
    for (int i = begin; i < end; i++) {
      if (in[i] != -1) {
        out[outpos++] = in[i];
      }
    }
  }

  std::cout << "Filtered values: ";
  for (int i = 0; i < N && i < 20; i++)
    std::cout << out[i] << " ";
  std::cout << "\n";
}

Overwriting /home/OpenMP/filter_pack.cpp


Compile and run:

In [ ]:
!g++ filter_pack.cpp -fopenmp -o filter_pack
!./filter_pack

Filtered values: 0 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 


### **Sparse Matrix-Vector-Multiply**

In a sparse matrix, most entries are zero. So, instead of storing all $N \times N$ entries, we store only the non-zero ones.
<br>
The most common format to do this is [**CSR (Compressed Sparse Row)**](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)):

- `k`: `0..(# non-zero elements)-1` index of a non-zero element
- `values[k]`: array of the non-zero elements, stored contigously row after row
- `columnidx[k]`: index of the column to which the `k`th non-zero element belongs
- `i`: `0..N-1` row index
- `rowptr[i]`: index of `values` (`k`) where the `i`th row begins

*A typical access pattern: from `rowptr` you get the initial `k` for the `i`-th row, then, for each `value` in `k`, `k + 1`, ... you check which column it belongs to `columnidx`.*

---

Here we want to perform a sparse matrix-vector product, where we assume the vector to be dense and the matrix sparse.
$$
res_i = \sum_{j = 0}^{N - 1} M_{i j} \cdot vec_j
$$

The idea is for every row (= output element) to GATHER data from arbitrary indices in the input vector `vec` according to the matrix's existing entries.
That is, for each position in a row of the matrix not holding a zero, we gather the corresponding `vec` element and carry out the multiply and accumulate.

In fact, working on row `i` of a sparse matrix requires this loop:
```c
for (int k = rowptr[i]; k < rowptr[i+1]; k++) {
  int col = columnidx[k];
  float val = values[k]; // M[i][col]
  float input = vec[col];
}
```
Where we clearly "gather" `vec[col]` at arbitrary column positions for each non-zero.

In [ ]:
%%writefile /home/OpenMP/spmv.cpp
#include <iostream>
#include <vector>
#include <random>
#include <omp.h>

#include "timer.hpp"

struct CSR {
  int n;
  std::vector<double> values;
  std::vector<int> columnidx;
  std::vector<int> rowptr;
};

// build a random sparse matrix in CSR format
// - n : number of rows/columns
// - nnz_per_row : number of non-zeros per row
CSR random_csr(int n, int nnz_per_row) {
  CSR M;
  M.n = n;
  M.rowptr.resize(n + 1);

  std::mt19937 rng(12345);
  std::uniform_int_distribution<int> columndist(0, n - 1);
  std::uniform_real_distribution<double> values_dense(-1.0, 1.0);

  M.values.reserve(n * nnz_per_row);
  M.columnidx.reserve(n * nnz_per_row);

  int k = 0;
  for (int i = 0; i < n; i++) {
    M.rowptr[i] = k;
    for (int t = 0; t < nnz_per_row; t++) {
      int col = columndist(rng);
      M.columnidx.push_back(col);
      M.values.push_back(values_dense(rng));
      k++;
    }
  }
  M.rowptr[n] = k;

  return M;
}

// parallel SpMV
void spmv(const CSR& M, const std::vector<double>& vec, std::vector<double>& res) {
  int n = M.n;
  res.assign(n, 0.0);

  // parallel over rows
  #pragma omp parallel for schedule(static)
  for (int i = 0; i < n; i++) {

    double sum = 0.0;
    int start = M.rowptr[i];
    int end = M.rowptr[i + 1];

    // GATHER vector entries for the row's columns
    #pragma omp parallel for schedule(static) reduction(+:sum)
    for (int k = start; k < end; k++) {
      int col = M.columnidx[k]; // index from which to gather
      sum += M.values[k] * vec[col];
    }

    res[i] = sum;
  }
}

int main() {
  int n = 2000;
  int nnz_per_row = 10;

  CSR M = random_csr(n, nnz_per_row);

  std::vector<double> vec(n), res;

  for (int i = 0; i < n; i++)
    vec[i] = (double)i * 0.001;

  double t0 = omp_get_wtime();
  time(spmv, M, vec, res);
  double t1 = omp_get_wtime();

  std::cout << "y[0] = " << res[0] << "\n";
  std::cout << "y[last] = " << res.back() << "\n";
}

Overwriting /home/OpenMP/spmv.cpp


Compile and ruin - ah, ehm... no - run:

In [ ]:
!g++ spmv.cpp -fopenmp -o spmv
!./spmv

[ function spmv took 1ms ]
y[0] = -0.334976
y[last] = 1.00368


### **Image Upsampling - Gather vs Scatter**

Consider the task of upsampling a B&W image, e.g. doubling its width and height while retaining its content (visually speaking).
<br>
In practice, this means computing each pixel in the new image from pixels of the original. For us, each original pixel will be copied over to its transposed coordinates and then blur a 1-pixed radius (3x3 area) around himself by averaging its value with that of nearby pixels doing the same thing.

There are two patterns we could think of to parallelize this:
- each input pixel goes to write himself its 3x3 target output area: SCATTER
- each output pixel goes fetch the input pixels to compute its value: GATHER

Which approach do you think is better?
<!--answer: see comments after the code-->
For now, let's implement both!

<br>

*Another way to see the two:*
- *GATHER: parallelize along output dimensions, each output pixel reads its input(s)*
- *SCATTER: parallelize along input dimensions, each input writes its output(s)*

In [ ]:
%%writefile /home/OpenMP/upsample.cpp
#include <cmath>
#include <vector>
#include <iomanip>
#include <iostream>
#include <omp.h>

#include "timer.hpp"

// SIMPLE GATHER: each output pixel reads from its nearest neighbor input pixels
// NOTE: this is not used here, but it's just to show how simple a gether can get in this case
void gather_upsample_easy(const std::vector<float>& input, int H, int W, std::vector<float>& output, int H2, int W2) {
  #pragma omp parallel for collapse(2)
  for (int i = 0; i < H2; i++) {
    for (int j = 0; j < W2; j++) {
      int src_i = i / 2;
      int src_j = j / 2;
      output[i * W2 + j] = input[src_i * W + src_j];
    }
  }
}

// GATHER: each output pixel reads from input pixels and averages its 1, 2, or 4 nearest neighbors
void gather_upsample(const std::vector<float>& input, int H, int W, std::vector<float>& output, int H2, int W2) {
  #pragma omp parallel for collapse(2)
  for (int oi = 0; oi < H2; oi++) {
    for (int oj = 0; oj < W2; oj++) {
      float sum = 0.0f;
      // visit a 3x3 output area
      for (int di = -1; di <= 1; di++) {
        for (int dj = -1; dj <= 1; dj++) {
          // keep only output elements corresponding to a new input
          if ((oi - di) % 2 != 0) continue;
          if ((oj - dj) % 2 != 0) continue;
          int i = (oi - di) / 2;
          int j = (oj - dj) / 2;
          if (i < 0 || i >= H || j < 0 || j >= W) continue;
          float v = input[i * W + j];
          // average of 1, 2, or 4 inputs, depending on the
          // positions in the 3x3 area where they are found
          sum += v / std::pow(2.0f, float(di*di + dj*dj));
        }
      }

      output[oi * W2 + oj] = sum;
    }
  }
}

// SCATTER: each input pixel spreads into a 3x3 region of output
void scatter_upsample(const std::vector<float>& input, int H, int W, std::vector<float>& output, int H2, int W2) {
  std::fill(output.begin(), output.end(), 0.0f);

  #pragma omp parallel for collapse(2)
  for (int i = 0; i < H; i++) {
    for (int j = 0; j < W; j++) {
      float v = input[i * W + j];
      int oi = 2 * i;
      int oj = 2 * j;
      // visit a 3x3 output area
      for (int di = -1; di <= 1; di++) {
        for (int dj = -1; dj <= 1; dj++) {
          int ii = oi + di;
          int jj = oj + dj;
          if (ii >= 0 && ii < H2 && jj >= 0 && jj < W2) {
            #pragma omp atomic
            output[ii * W2 + jj] += v / std::pow(2.0f, dj*dj + di*di);
          }
        }
      }
    }
  }
}

int main() {
  int H = 600, W = 800;
  int H2 = H * 2, W2 = W * 2;

  std::vector<float> input(H * W);
  std::vector<float> output(H2 * W2);

  for (int i = 0; i < H; i++)
    for (int j = 0; j < W; j++)
      input[i * W + j] = i * 1.4f + j;

  std::cout << "Input:\n";
  for (int i = 0; i < H && i < 5; i++) {
    for (int j = 0; j < W && j < 5; j++)
      std::cout << std::setw(6) << std::fixed << std::setprecision(2) << std::right << input[i * W + j] << " ";
    std::cout << "\n";
  }

  std::cout << "Gather upsampling...\n";
  time(gather_upsample, input, H, W, output, H2, W2);

  std::cout << "Output:\n";
  for (int i = 0; i < H2 && i < 10; i++) {
    for (int j = 0; j < W2 && j < 10; j++)
      std::cout << std::setw(6) << std::fixed << std::setprecision(2) << std::right << output[i * W2 + j] << " ";
    std::cout << "\n";
  }

  std::cout << "Scatter upsampling...\n";
  time(scatter_upsample, input, H, W, output, H2, W2);

  std::cout << "Output:\n";
  for (int i = 0; i < H2 && i < 10; i++) {
    for (int j = 0; j < W2 && j < 10; j++)
      std::cout << std::setw(6) << std::fixed << std::setprecision(2) << std::right << output[i * W2 + j] << " ";
    std::cout << "\n";
  }
}

Overwriting /home/OpenMP/upsample.cpp


Just by glancing at the code, the GATHET can be implemented very cleanly, while the SCATTER is far more troublesome!

That is because:
- in the GATHER, each output pixel determines which input pixel(s) it needs, thus each thread writes to its own output location: safe, race-free.

- in the SCATTER, each input pixel distributes its value into multiple output pixels, multiple threads may end up writing to the same output location: we must use atomics.

Compile and run:

In [ ]:
!g++ upsample.cpp -fopenmp -o upsample
!./upsample

Input:
  0.00   1.00   2.00   3.00   4.00 
  1.40   2.40   3.40   4.40   5.40 
  2.80   3.80   4.80   5.80   6.80 
  4.20   5.20   6.20   7.20   8.20 
  5.60   6.60   7.60   8.60   9.60 
Gather upsampling...
[ function gather_upsample took 130ms ]
Output:
  0.00   0.50   1.00   1.50   2.00   2.50   3.00   3.50   4.00   4.50 
  0.70   1.20   1.70   2.20   2.70   3.20   3.70   4.20   4.70   5.20 
  1.40   1.90   2.40   2.90   3.40   3.90   4.40   4.90   5.40   5.90 
  2.10   2.60   3.10   3.60   4.10   4.60   5.10   5.60   6.10   6.60 
  2.80   3.30   3.80   4.30   4.80   5.30   5.80   6.30   6.80   7.30 
  3.50   4.00   4.50   5.00   5.50   6.00   6.50   7.00   7.50   8.00 
  4.20   4.70   5.20   5.70   6.20   6.70   7.20   7.70   8.20   8.70 
  4.90   5.40   5.90   6.40   6.90   7.40   7.90   8.40   8.90   9.40 
  5.60   6.10   6.60   7.10   7.60   8.10   8.60   9.10   9.60  10.10 
  6.30   6.80   7.30   7.80   8.30   8.80   9.30   9.80  10.30  10.80 
Scatter upsampling...
[ function s

### **Expand**

Say you are developing a text editor. Yes, why not!
<br>
You need to deal with a very large amount of text, and worst of all, this text could undergo updates, insertions, and deletions, at any arbitrary point.
<br>
To deal with this, a very elementary solution would be to subdivide the text into a double-linked list of constant-sized arrays.
If you can make it so that each array has a bit of extra free space at the end, you can then immediately accomodate small arbitrary updates, and only then dynamically alter the linked list structure to enable larger changes.
<br>
<br>
Then, just as with any coding project for "personal use", you had an evil genius "I hate myself so much for doing this" moment.
Spaces are costly, and tabs are annoying, let's therefore store any space as two consecutive bytes, one for the space character itself, the other for the number of spaces it truly represents.
<br>
<br>
Now you have a problem tho, your file's content, that should be a continous string, is now fragmented into several arrays, each with arbitrary amounts of padding at the end, and with a notation for spaces not compatible with any other "normal" text editor.
<br>
When it is time to save the file, we need to concatenate all the valid parts of the arrays into a single one and expand spaces.
<br>
This task turns out to be quite easily parallelizable!

We can use the EXPAND pattern, the fusion of MAP and PACK, where the mapped function can return 0, 1, or more elements and the pack concatenates all such returned values.

In our case:
- mapped function: iterate each array and prepare its final string;
- pack: SCAN of the number of elements returned by each array, then SCATTER them into the final array;

*Note: as already showed in [Linked List Traversal](#linked-list), going over linked lists in parallel isn't that efficient, but in this case it could be worth it to accelerate the ensuing EXPAND pattern.*

In [ ]:
%%writefile /home/OpenMP/text_expand.cpp
#include <iostream>
#include <vector>
#include <string>
#include <cstring>
#include <cstdlib>
#include <ctime>
#include <omp.h>

#define BLOCK_SIZE 64 // maximum size of each block's array

// one of the arrays in the linked list
struct Block {
  char data[BLOCK_SIZE];
  int size; // number of valid characters
  Block* next;
  Block* prev;
  Block() : size(0), next(nullptr), prev(nullptr) {}
};

// initialize the linked list from a custom multi-line string
Block* build_list_from_text() {
  std::string input =
R"(
int main() {
  int N = 1000000;

  double *A = (double*) malloc(N * sizeof(double));
  double *B = (double*) malloc(N * sizeof(double));

  // initialize vectors
  for (int i = 0; i < N; i++) {
    A[i] = i * 0.001;
    B[i] = (N - i) * 0.002;
  }
                       //HOW MUCH I LOVE SPACES
  double dot = 0.0;
  double start_time = omp_get_wtime();

  // parallel reduce
  #pragma omp parallel for reduction(+ : dot) schedule(static)
  for (int i = 0; i < N; i++) {
    dot += A[i] * B[i];
  }
                       //THIS SPACES ARE SO EFFICIENT NOW
  double end_time = omp_get_wtime();

  printf("Dot product = %.5f\n", dot);
  printf("Computed in %.5f seconds using %d threads.\n",
  end_time - start_time, omp_get_max_threads());
                       //SPAAAAAAACE! <robot-like voice>
  free(A);
  free(B);
}
)";

  if (input.empty()) return nullptr;

  // helper: append one byte to the current block, allocating a new one if needed
  auto append_byte = [](Block*& current, char byte) {
    if (current->size == BLOCK_SIZE) {
      Block* nxt = new Block();
      current->next = nxt;
      nxt->prev = current;
      current = nxt;
    }
    current->data[current->size++] = byte;
  };

  Block* head = new Block();
  Block* current = head;

  // encode spaces: a run of k spaces => ' ' + (unsigned char)k
  for (size_t i = 0; i < input.size(); ++i) {
    char c = input[i];
    if (c == ' ') {
      size_t j = i;
      while (j < input.size() && input[j] == ' ')
        ++j;
      size_t run_len = j - i;
      while (run_len > 0) {
        unsigned char chunk = static_cast<unsigned char>(
            std::min(run_len, static_cast<size_t>(255)));
        append_byte(current, ' ');
        append_byte(current, static_cast<char>(chunk));
        run_len -= chunk;
      }
      i = j - 1;
    } else {
      append_byte(current, c);
    }
  }
  return head;
}


// mapped function: pre-process a block
// => isolate the string and convert "space + count" into repeated spaces
std::string expand_block(const Block* blk) {
  std::string out;
  out.reserve(blk->size * 2); // rough guess

  for (int i = 0; i < blk->size; ++i) {
    char c = blk->data[i];
    if (c == ' ' && i + 1 < blk->size) {
      unsigned count = static_cast<unsigned char>(blk->data[i + 1]);
      out.append(count, ' ');
      i++; // skip the count byte
    } else {
      out.push_back(c);
    }
  }
  return out;
}

int main() {
  Block* head = build_list_from_text();

  // linked lists are a pain:
  // collect block pointers in a vector for quicker indexing
  std::vector<Block*> blocks;
  for (Block* p = head; p != nullptr; p = p->next)
    blocks.push_back(p);

  int n = blocks.size();
  if (n == 0) {
    std::cout << "" << std::endl;
    return 0;
  }

  // MAP: expand each block independently
  std::vector<std::string> expanded(n);
  std::vector<size_t> sizes(n);
  #pragma omp parallel for
  for (int i = 0; i < n; ++i) {
    expanded[i] = expand_block(blocks[i]);
    sizes[i] = expanded[i].size();
  }

  // INCLUSIVE SCAN: prefix sum of sizes
  // NOTE: to keep the code brief, this is done sequentially, after all we have seen the scan plenty of times already!
  std::vector<size_t> prefix(n + 1, 0);
  for (int i = 0; i < n; ++i)
    prefix[i + 1] = prefix[i] + sizes[i];
  size_t total = prefix[n];

  // SCATTER: write into the final buffer
  std::string final_text(total, '\0');
  #pragma omp parallel for
  for (int i = 0; i < n; ++i) {
    size_t start = prefix[i];
    std::memcpy(final_text.data() + start, expanded[i].data(), sizes[i]);
  }

  std::cout << final_text << std::endl;
  return 0;
}

Overwriting /home/OpenMP/text_expand.cpp


Compile and run:

In [ ]:
!g++ text_expand.cpp -fopenmp -o text_expand
!./text_expand


int main() {
  int N = 1000000;

  double *A = (double*) malloc(N * sizeof(double));
  double *B = (double*) malloc(N * sizeof(double));

  // initialize vectors
  for (int i = 0; i < N; i++) {
    A[i] = i * 0.001;
    B[i] = (N - i) * 0.002;
  }
                       //HOW MUCH I LOVE SPACES
  double dot = 0.0;
  double start_time = omp_get_wtime();

  // parallel reduce
  #pragma omp parallel for reduction(+ : dot) schedule(static)
  for (int i = 0; i < N; i++) {
    dot += A[i] * B[i];
  }
                       //THIS SPACES ARE SO EFFICIENT NOW
  double end_time = omp_get_wtime();

  printf("Dot product = %.5f\n", dot);
  printf("Computed in %.5f seconds using %d threads.\n",
  end_time - start_time, omp_get_max_threads());
                       //SPAAAAAAACE! <robot-like voice>
  free(A);
  free(B);
}



## **Parallelizing Loops**

As you have analyzed in the [Loop Dependencies](#loop-dependencies) section, when we have inter-iteration data dependencies there is often still a bit of parallelism to squeeze out of loopnests. Let's now see that in practice.

<img src="https://i.imgur.com/2euHumY.png" align="right" width="450px" border="2"> In general: a **recurrence pattern** is a type of loop where the calculation for one iteration depends on the resuls of a previous iteration.

To identify a viable parallelization direction, the trick is to find a plane that cuts through the grid of intermediate results so that all references to previously computed values are on one side of the plane.
Such a plane is called a *separating hyperplane*.
You can then sweep (iterate) through the data in a direction perpendicular to this plane and perform all the operations on the plane at each iteration in parallel.
<br>
*Note: you can visualize it as moving the plan along its normal direction at each global iteration, then going in parallel "inside" the plane.*

It can be proven that an hyperplane can always be found if the dependencies are constant offsets.
See "[The parallel execution of DO loops](https://dl.acm.org/doi/10.1145/360827.360844)", Laslie Lamport.

### **Exercise 15 - Spot the Hyperplane**

Given the following loop-nests, identify their data dependencies and identify the slope of the separating hyperplane along which the computation can be parallelized.

For convenience, let use say that $i$ spans the rows and $j$ the columns.

<!--
Loopnest 1)
Imagining the computation to go from the top-left towards the bottom-right of a grid, data dependencies come in from the left and top of every cell, at distance 1.
The hyperplane (a line, here) has angular coefficient m = 1.

Loopnest 2)
Imagining the computation to go from the top-left towards the bottom-right of a grid, data dependencies come in from the left and top-right of every cell, at distance 1.
The hyperplane (a line, here) has angular coefficient m = 1/2.
You can read this as "for every cell I move downward, I need to go 2 to the left to stay within dependencies".

Loopnest 3)
Imagining the computation to go from the top-left towards the bottom-right of a grid, data dependencies come in from the left of every cell at distance 1, and top of every cell at distance 2.
The hyperplane (a line, here) has angular coefficient m = 2.

Loopnest 4)
Imagining the computation to go from the top-left-back towards the bottom-right-front of a grid, data dependencies come in from the left, top, and back of every cell, at distance 1.
The hyperplane (a 2D plane, here) has both angular equal 1.


In general, higher angular coefficients mean that more rows can run in parallel, while lower ones imply that more columns can run in parallel.
-->

In [ ]:
%%writefile highlight_me.cpp

// Loopnest 1)
for (j = 0; j < 100; j++)
    for (i = 0; i < 100; i++)
        a[i][j] = f(a[i][j - 1], a[i - 1][j]);

// Loopnest 2)
for (j = 0; j < 100; j++)
    for (i = 0; i < 100; i++)
        a[i][j] = f(a[i][j - 1], a[i - 1][j + 1]);

// Loopnest 3)
for (j = 0; j < 100; j++)
    for (i = 0; i < 100; i++)
        a[i][j] = f(a[i][j - 1], a[i - 2][j]);

// Loopnest 4)
for (j = 0; j < 100; j++)
    for (i = 0; i < 100; i++)
        for (k = 0; k < 20; k++)
            a[i][j][k] = f(
              a[i][j - 1][k], a[i - 1][j + 1][k],
              a[i][j][k - 1], a[i][j][k]
            );

### **Recurrence Pattern**

Consider a 5-points STENCIL pattern.
It can be seen as a MAP where the output depends on multiple, neighboring, inputs!

It is typically used in simulations (e.g. automata) where it updates an entire grid in discrete time steps. To do this sequentially, we can use a single grid instance, but to go parallel often two are used in a "ping-pong" fashion, a copy is read-only and serves as the previous state from which to write the next state in the other copy, then the two are swapped and the process repeats for the next time step.

But what if the grid is massive, or we simply can't afford to have multiple copies? We need **in-place** updates.
And because of this, must we go serial now? Well, the loop often looks like this:
```
for i in [1, N-1):
  for j in [1, M-1):
    g[i][j] = f(g[i][j], g[i-1][j], g[i][j-1], g[i+1][j], g[i][j+1])
```
<img src="https://www.eecg.utoronto.ca/~tsa/jasmine/blocksched1.gif" align="right" width="200px"> Well, taking a lesson from dynamic programming, we can use what is called **wavefront**: sweep the computation diagonally!
<br>
In particular, for any inter-iteration, fixed-offset, dependency, there is always a skew for which a wavefront can propagate along diagonals, running in parallel inside each diagonal.

A good source on the subject: https://www.eecg.utoronto.ca/~tsa/jasmine/wave.html

An important distinction to be made while look at the code as if it ran sequentially:
- `g[i][j], g[i+1][j], g[i][j+1]` are anti-dependencies, old values are read when the code runs serially, they don't constitute a problem unless we are "too much ahead";
- `g[i-1][j], g[i][j-1]` are true dependencies, signifying that the present computation requires, new, up-to-date values from previous ones;

---

In this example we step forward a 7-ponts STENCIL computation in-place. It is a 5-points stencil, but we also go 2-right and 2-left of the current cell.

For parallelization, we have a first sequential loop over diagonals, and then a parallel loop along each diagonal.

Be wary: the diagonal varies a log in span throughout the computation, and so does the amount of work we can give to processes. Load balancing must be kept in mind here!

In [ ]:
%%writefile /home/OpenMP/wavefront.cpp
#include <iostream>
#include <iomanip>
#include <algorithm>
#include <vector>
#include <omp.h>

constexpr int N = 512;

int main() {
  std::vector<std::vector<double>> g(N, std::vector<double>(N, 0.0));

  // initial state (every fifth row at "i")
  for (int i = 0; i < N; i += 5)
    for (int j = 0; j < N; j++)
      g[i][j] = (float)(i);

  std::cout << "Initial:\n";
  for (int i = 0; i < N && i < 20; i++) {
    for (int j = 0; j < N && j < 20; j++)
      std::cout << std::setw(5) << std::fixed << std::setprecision(2) << std::right << g[i][j] << " ";
    std::cout << "\n";
  }

  // wavefront along diagonals (from top-left to bottom-right)
  for (int d = 2; d <= 2*(N - 3); d++) {

    #pragma omp parallel for
    for (int j = 2; j < N-2; j++) {
      int i = d - j;
      if (i < 2 || i >= N-2) continue;
      double newv = g[i - 1][j] + g[i + 1][j] + g[i][j - 1] + g[i][j + 1] + g[i][j - 2] + g[i][j + 2] + g[i][j];
      g[i][j] = newv / 7.0;
    }
  }

  std::cout << "Final:\n";
  for (int i = 0; i < N && i < 20; i++) {
    for (int j = 0; j < N && j < 20; j++)
      std::cout << std::setw(5) << std::fixed << std::setprecision(2) << std::right << g[i][j] << " ";
    std::cout << "\n";
  }
}

Overwriting /home/OpenMP/wavefront.cpp


Compile and run:

In [ ]:
!g++ wavefront.cpp -fopenmp -o wavefront
!./wavefront

Initial:
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00  5.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00

### **Successive Over-Relaxations**

Successive Over-Relaxations (SOR) is an iterative method for numerically solving grid-based Partial Differential Equations (PDEs) (e.g. finite-difference discretizations of Laplace/Poisson).
<br>
It is essentially the Gauss-Seidel iterative method, but each update is multiplied by a relaxation parameter $\omega \in (0, 2)$:
$$
u^{(new)} = (1 - \omega) \cdot u^{(old)} + \omega \cdot u^{(GS)}
$$
where $u^{(GS)}$ is the value you would compute with standard Gauss-Seidel method.

Advantage:
<br>
Our input isn't made of only old values, SOR uses updated values immediately as they become available.
Thus the grid can be updated **in-place**, and we can thus get away with a single copy of it!
De facto, the update uses the latest information in a fixed order (sweep).

For everyone unfamiliar: everything up to now comes down to a 4-points average STENCIL (5-points minus the center).
However, we can leverage the properties of the problem to know that we can update, in a staggered fashion, one cell every two at each time step.
And an acceptable solution will still be reached.
In other words, the 4-points stencil makes the grid-wise dependency graph bipartite.
This removes our data dependencies and gives us one fully parallel iteration every two steps.

<img src="https://wallpapers.com/images/hd/red-black-background-pomifr0hf37fyulf.jpg" border="2" align="right" width="200px">

SOR works when:
- the update stencil is monotone (e.g. average)
- the iteration order is consistent (e.g. left-to-right, top-to-bottom)
- there are no data races (this we need to enforce with our code!)

How to parallelize: red-black coloring!
<br>
Red-black SOR divides the grid like a chessboard, with diagonally adjacent cells all of the same color, either red or black (Note: the specific scheme depends on the stencil's shape).
First, only red cells are updated in parallel (all black neighbors are thus safe to read).
Then, we update black cells in parallel (all red neighbors are thus safe).
And repeat.
Ultimately, the stencil never reads a value being written in the same step.

In [ ]:
%%writefile /home/OpenMP/sor.cpp
#include <stdio.h>
#include <math.h>
#include <omp.h>

#define N 256
#define MAX_ITERS 5000
#define TOL 1e-4

int main() {
  static double u[N][N] = {0};
  const double omega = 1.8;

  // example: boundary condition on the top row
  for (int j = 0; j < N; j++)
    u[0][j] = 1.0;

  for (int iter = 0; iter < MAX_ITERS; iter++) {
    double maxdiff = 0.0;

    // RED step
    #pragma omp parallel for reduction(max:maxdiff)
    for (int i = 1; i < N-1; i++)
      for (int j = 1 + (i & 1); j < N-1; j += 2) {
        double old = u[i][j];
        double avg = 0.25 * (u[i - 1][j] + u[i + 1][j] + u[i][j - 1] + u[i][j + 1]);
        double newv = (1.0 - omega)*old + omega*avg;
        u[i][j] = newv;
        maxdiff = fmax(maxdiff, fabs(newv - old));
      }

    // BLACK step
    #pragma omp parallel for reduction(max:maxdiff)
    for (int i = 1; i < N-1; i++)
      for (int j = 1 + ((i+1) & 1); j < N-1; j += 2) {
        double old = u[i][j];
        double avg = 0.25 * (u[i - 1][j] + u[i + 1][j] + u[i][j - 1] + u[i][j + 1]);
        double newv = (1.0 - omega)*old + omega*avg;
        u[i][j] = newv;
        maxdiff = fmax(maxdiff, fabs(newv - old));
      }

    if (maxdiff < TOL) {
      printf("Converged after %d iterations (diff=%.6f)\n", iter, maxdiff);
      break;
    }
  }

  printf("u[64][64] = %f\n", u[64][64]);
  return 0;
}


Overwriting /home/OpenMP/sor.cpp


Compile and run:

In [ ]:
!g++ sor.cpp -fopenmp -o sor
!./sor

Converged after 1279 iterations (diff=0.000100)
u[64][64] = 0.393263


## **Inspect The Hardware**

See what the machine we are using (here on Colab) is capable of:

In [ ]:
%%writefile /home/OpenMP/inspect_hw.cpp
#include <iostream>
#include <fstream>
#include <string>
#include <omp.h>
#include <unistd.h>
#include <thread>

// helper to read cache info from Linux sysfs
long read_cache_size(int level) {
    std::string path = "/sys/devices/system/cpu/cpu0/cache/index" + std::to_string(level) + "/size";
    std::ifstream file(path);
    if (!file.is_open()) return -1;
    std::string value;
    file >> value;
    long size = std::stol(value);
    if (value.find('K') != std::string::npos) size *= 1024;
    if (value.find('M') != std::string::npos) size *= 1024 * 1024;
    return size;
}

int main() {
  // CPU and threading info
  int omp_procs = omp_get_num_procs();
  int omp_max_threads = omp_get_max_threads();
  unsigned int hw_threads = std::thread::hardware_concurrency();

  std::cout << "=== System Info (OpenMP + Hardware) ===\n";
  std::cout << "Logical processors available (OpenMP): " << omp_procs << "\n";
  std::cout << "Max OpenMP threads: " << omp_max_threads << "\n";
  std::cout << "Hardware concurrency (std::thread): " << hw_threads << "\n";

  // hyperthreading available if hardware threads > physical cores
  if (hw_threads > omp_procs / 2)
    std::cout << "Hyperthreading likely enabled.\n";
  else
    std::cout << "No hyperthreading detected (or not applicable).\n";

  // cache info
  for (int i = 0; i < 3; ++i) {
    long size = read_cache_size(i);
    if (size > 0)
      std::cout << "L" << (i + 1) << " cache size: " << size / 1024 << " KB\n";
  }

  // RAM info
  long pages = sysconf(_SC_PHYS_PAGES);
  long page_size = sysconf(_SC_PAGE_SIZE);
  double total_ram_gb = (double)pages * page_size / (1024.0 * 1024.0 * 1024.0);
  std::cout << "Total physical memory: " << total_ram_gb << " GB\n";

  // OpenMP runtime confirmation
  #pragma omp parallel
  {
    #pragma omp single
    std::cout << "Actual threads used by default by OpenMP: " << omp_get_num_threads() << "\n";
  }

  return 0;
}


Overwriting /home/OpenMP/inspect_hw.cpp


Compile and run:

In [ ]:
!g++ inspect_hw.cpp -fopenmp -o inspect_hw
!./inspect_hw

=== System Info (OpenMP + Hardware) ===
Logical processors available (OpenMP): 2
Max OpenMP threads: 2
Hardware concurrency (std::thread): 2
Hyperthreading likely enabled.
L1 cache size: 32 KB
L2 cache size: 32 KB
L3 cache size: 256 KB
Total physical memory: 12.6714 GB
Actual threads used by default by OpenMP: 2
